In [71]:
from prediction_utils.cohorts.cohort import BQCohort


class SepsisCohort(BQCohort):
    def get_base_query(self, format_query=True):
        query = """ (
        SELECT * FROM (
            SELECT 
                t1.person_id, 
                visit_concept_id, 
                visit_start_datetime, 
                visit_end_datetime
            FROM {dataset_project}.{dataset}.visit_occurrence t1
            INNER JOIN {dataset_project}.{dataset}.person as t2
                ON t1.person_id = t2.person_id
            WHERE
                visit_concept_id in (9201, 262)
                AND visit_end_datetime > visit_start_datetime
                AND visit_end_datetime is not NULL
                AND visit_start_datetime is not NULL
        )
        {where_str}
        {limit_str}
        )
        """
        if not format_query:
            return query
        else:
            return query.format_map(self.config_dict)

    def get_transform_query(self, format_query=True):
        query = """
            WITH visits AS (
              SELECT *
              FROM {base_query}
            ),
            visits_melt AS (
              SELECT person_id, visit_start_datetime AS endpoint_date, 1 as endpoint_type
              FROM visits
              UNION ALL
              SELECT person_id, visit_end_datetime AS endpoint_date, -1 as endpoint_type
              FROM visits
            ),
            counts1 AS (
              SELECT *, COUNT(*) * endpoint_type as count
              FROM visits_melt
              GROUP BY person_id, endpoint_date, endpoint_type
            ),
            counts2 AS (
              SELECT person_id, endpoint_date, SUM(count) as count
              FROM counts1
              GROUP BY person_id, endpoint_date
            ),
            counts3 AS (
              SELECT person_id, endpoint_date,
                  SUM(count) OVER(PARTITION BY person_id ORDER BY endpoint_date) as count
              FROM counts2
            ),
            cum_counts AS (
              SELECT *, ROW_NUMBER() OVER(PARTITION BY person_id ORDER BY endpoint_date) as row_number
              FROM counts3
            ),
            discharge_times AS (
              SELECT person_id, endpoint_date, 'discharge_date' as endpoint_type, row_number
              FROM cum_counts
              WHERE count = 0
            ),
            discharge_times_row_shifted AS (
              SELECT person_id, (row_number + 1) as row_number
              FROM discharge_times
            ),
            first_admit_times AS (
              SELECT person_id, endpoint_date, 'admit_date' as endpoint_type
              FROM cum_counts
              WHERE row_number = 1
            ),
            other_admit_times AS (
              SELECT t1.person_id, endpoint_date, 'admit_date' as endpoint_type
              FROM cum_counts t1
              INNER JOIN discharge_times_row_shifted AS t2
              ON t1.person_id=t2.person_id AND t1.row_number=t2.row_number
            ),
            aggregated_endpoints AS (
              SELECT person_id, endpoint_date, endpoint_type
              FROM discharge_times
              UNION ALL
              SELECT person_id, endpoint_date, endpoint_type
              FROM first_admit_times
              UNION ALL
              SELECT person_id, endpoint_date, endpoint_type
              FROM other_admit_times
            ),
            result_long AS (
              SELECT *, ROW_NUMBER() OVER(PARTITION BY person_id, endpoint_type ORDER BY endpoint_date) as row_number
              FROM aggregated_endpoints
            ),
            discharge_times_final AS (
                SELECT person_id, endpoint_date as discharge_date, row_number
                FROM result_long
                WHERE endpoint_type = 'discharge_date'
            ),
            admit_times_final AS (
                SELECT person_id, endpoint_date as admit_date, row_number
                FROM result_long
                WHERE endpoint_type = 'admit_date'
            ),
            result AS (
                SELECT t1.person_id, admit_date, discharge_date, t1.row_number
                FROM admit_times_final t1
                INNER JOIN discharge_times_final as t2
                ON t1.person_id=t2.person_id AND t1.row_number=t2.row_number
            )
            SELECT person_id, admit_date, discharge_date
            FROM result
            ORDER BY person_id, row_number
        """

        if not format_query:
            return query
        else:
            return query.format_map(
                {**self.config_dict, **{"base_query": self.get_base_query()}}
            )

    def get_create_query(self, format_query=True):

        query = """ 
            CREATE OR REPLACE TABLE {rs_dataset_project}.{rs_dataset}.{cohort_name} AS
            {query}
        """

        if not format_query:
            return query
        else:
            return query.format_map(
                {**self.config_dict, **{"query": self.get_transform_query()}}
            )

    def get_suspected_infection(self, format_query=True):
        query = """
            WITH 
            blood_culture_list AS (
                SELECT 
                    descendant_concept_id AS concept_id
                FROM {dataset_project}.{dataset}.concept_ancestor
                WHERE ancestor_concept_id = 4107893
            ),
            blood_culture_from_measurement_via_ancestor AS (  
                SELECT 
                    person_id, 
                    measurement_DATETIME
                FROM {dataset_project}.{dataset}.measurement AS measure
                WHERE measure.measurement_concept_id IN (
                    SELECT concept_id
                    FROM blood_culture_list)
            ),
            systemic_abx_list AS (
                SELECT 
                    descendant_concept_id AS concept_id
                FROM {dataset_project}.{dataset}.concept_ancestor
                WHERE ancestor_concept_id = 21602796 
            ),
            systemic_abx_from_drug_exposure_via_ancestor AS (
                SELECT 
                    person_id, 
                    drug_concept_id, 
                    drug_exposure_start_DATETIME
                FROM {dataset_project}.{dataset}.drug_exposure AS drug
                WHERE drug.drug_concept_id IN (
                    SELECT concept_id
                    FROM systemic_abx_list)
            ),
            systemic_abx_from_drug_exposure_with_name AS (
                SELECT 
                    systemic_abx.*, 
                    concept.concept_name AS systemic_abx_type  
                FROM systemic_abx_from_drug_exposure_via_ancestor AS systemic_abx
                INNER JOIN {dataset_project}.{dataset}.concept AS concept
                ON systemic_abx.drug_concept_id = concept.concept_id
            ),
            bc_abx AS (
                SELECT 
                    blood_culture.person_id, 
                    blood_culture.measurement_DATETIME as bc_DATETIME,
                    systemic_abx.drug_exposure_start_DATETIME, 
                    systemic_abx.systemic_abx_type
                FROM blood_culture_from_measurement_via_ancestor AS blood_culture
                LEFT JOIN systemic_abx_from_drug_exposure_with_name AS systemic_abx
                ON blood_culture.person_id = systemic_abx.person_id
            ),
            admit_bc_abx AS (
                SELECT 
                    admission_rollup.*, 
                    bc_abx.bc_DATETIME, 
                    bc_abx.drug_exposure_start_DATETIME, 
                    bc_abx.systemic_abx_type   
                FROM {admission_rollup} as admission_rollup
                LEFT JOIN bc_abx AS bc_abx
                ON admission_rollup.person_id = bc_abx.person_id
            ),
            susp_inf_allrows AS (
                SELECT 
                    person_id, 
                    bc_DATETIME, 
                    drug_exposure_start_DATETIME, 
                    visit_start_datetime as admit_date, 
                    visit_end_datetime as discharge_date, 
                    systemic_abx_type,
                    datetime_diff(drug_exposure_start_DATETIME, bc_DATETIME, DAY) as days_bc_abx
                FROM admit_bc_abx as admit_bc_abx
                WHERE
                    CAST(bc_DATETIME AS DATE) >= DATE_SUB(visit_start_datetime, INTERVAL 1 DAY) AND  CAST(bc_DATETIME AS DATE) <= visit_end_datetime AND
                    CAST(drug_exposure_start_DATETIME AS DATE) >= DATE_SUB(visit_start_datetime, INTERVAL 1 DAY) AND  CAST(drug_exposure_start_DATETIME AS DATE) <= visit_end_datetime
                AND
                    CAST(bc_DATETIME AS DATE)<= CAST(DATETIME_ADD(drug_exposure_start_DATETIME, INTERVAL 1 DAY) AS DATE) AND
                    CAST(bc_DATETIME AS DATE)>= CAST(DATETIME_SUB(drug_exposure_start_DATETIME, INTERVAL 3 DAY) AS DATE) 
                ORDER BY person_id, visit_start_datetime, bc_DATETIME, drug_exposure_start_DATETIME
            )
            SELECT 
                person_id, 
                admit_date, 
                MIN(discharge_date) AS discharge_date,
                MIN(bc_DATETIME) as min_bc,
                MIN(drug_exposure_start_DATETIME) as min_systemic_abx,
                LEAST(MIN(bc_DATETIME),MIN(drug_exposure_start_DATETIME)) as index_date
            FROM susp_inf_allrows 
            GROUP BY person_id, admit_date
            ORDER BY person_id, admit_date 
            """

        if not format_query:
            return query
        else:
            return query.format_map(self.config_dict)

    def get_sepsis_cohort_initial(self, format_query=True):
        query = """
            WITH 
            platelet_from_measurement AS (
                SELECT 
                    measure.person_id, 
                    measure.measurement_DATETIME, 
                    measure.value_as_number, 
                    concept.concept_name AS measure_type  
                FROM {dataset_project}.{dataset}.measurement AS measure
                INNER JOIN {dataset_project}.{dataset}.concept AS concept
                ON measure.measurement_concept_id = concept.concept_id
                WHERE concept.concept_id in(
                    SELECT
                        c.concept_id
                    FROM {dataset_project}.{dataset}.concept c
                    WHERE c.concept_id in (37037425,40654106)
                    UNION DISTINCT
                    SELECT
                        c.concept_id
                    FROM {dataset_project}.{dataset}.concept c
                    INNER JOIN {dataset_project}.{dataset}.concept_ancestor ca
                    ON c.concept_id = ca.descendant_concept_id
                    AND ca.ancestor_concept_id in (37037425,40654106)
                    AND c.invalid_reason is null
                )
            ),
            bilirubin_from_measurement AS (
                SELECT 
                    measure.person_id, 
                    measure.measurement_DATETIME, 
                    measure.value_as_number, 
                    concept.concept_name AS measure_type  
                FROM {dataset_project}.{dataset}.measurement AS measure
                INNER JOIN {dataset_project}.{dataset}.concept AS concept
                ON measure.measurement_concept_id = concept.concept_id
                WHERE concept.concept_id in (
                    SELECT
                        c.concept_id
                    FROM {dataset_project}.{dataset}.concept c
                    WHERE c.concept_id in (3024128, 4230543)
                    UNION DISTINCT
                    SELECT
                        c.concept_id
                    FROM {dataset_project}.{dataset}.concept c
                    INNER JOIN {dataset_project}.{dataset}.concept_ancestor ca
                    ON c.concept_id = ca.descendant_concept_id
                    AND ca.ancestor_concept_id in (3024128, 4230543)
                    AND c.invalid_reason is null
                )
            ),
            ionotrope_list AS (
                SELECT 
                    descendant_concept_id AS concept_id
                FROM {dataset_project}.{dataset}.concept_ancestor
                WHERE ancestor_concept_id IN (21600284, 21600287, 21600303, 21600283, 21600308)
            ),
            ionotrope_from_drug_exposure_via_ancestor AS ( 
                SELECT *
                FROM {dataset_project}.{dataset}.drug_exposure AS drug
                WHERE drug.drug_concept_id IN (
                    SELECT concept_id
                    FROM ionotrope_list
                )
            ),
            ionotrope_from_drug_exposure_with_name AS (
                SELECT 
                    ionotrope.*, 
                    concept.concept_name AS ionotrope_type  
                FROM ionotrope_from_drug_exposure_via_ancestor AS ionotrope
                INNER JOIN {dataset_project}.{dataset}.concept AS concept
                ON ionotrope.drug_concept_id = concept.concept_id
            ),
            creatinine_from_measurement AS (
                SELECT 
                    measure.person_id, 
                    measure.measurement_DATETIME, 
                    measure.value_as_number, 
                    concept.concept_name AS measure_type  
                FROM {dataset_project}.{dataset}.measurement AS measure
                INNER JOIN {dataset_project}.{dataset}.concept AS concept
                ON measure.measurement_concept_id = concept.concept_id
                WHERE concept.concept_id in (
                    SELECT
                        c.concept_id
                    FROM {dataset_project}.{dataset}.concept c
                    WHERE c.concept_id in (37029387,4013964,2212294,3051825)
                    UNION DISTINCT
                    SELECT
                        c.concept_id
                    FROM {dataset_project}.{dataset}.concept c
                    INNER JOIN {dataset_project}.{dataset}.concept_ancestor ca
                    ON c.concept_id = ca.descendant_concept_id
                    AND ca.ancestor_concept_id in (37029387,4013964,2212294,3051825)
                    AND c.invalid_reason is null
                )
            ),
            lactate_from_measurement AS (
                SELECT 
                    measure.person_id, 
                    measure.measurement_DATETIME, 
                    measure.value_as_number, 
                    concept.concept_name AS measure_type  
                FROM {dataset_project}.{dataset}.measurement AS measure
                INNER JOIN {dataset_project}.{dataset}.concept AS concept
                ON measure.measurement_concept_id = concept.concept_id
                WHERE concept.concept_id = 3047181
                UNION ALL
                SELECT 
                    measure.person_id, 
                    measure.measurement_DATETIME, 
                    measure.value_as_number/9.0, 
                    concept.concept_name AS measure_type  
                FROM {dataset_project}.{dataset}.measurement AS measure /* Normalize the mg/dL measurement to mmol/L */
                INNER JOIN {dataset_project}.{dataset}.concept AS concept
                ON measure.measurement_concept_id = concept.concept_id
                WHERE concept.concept_id = 3020138
            ),
            paO2_from_measurement AS (
                SELECT 
                    measure.person_id, 
                    measure.measurement_DATETIME, 
                    measure.value_as_number, 
                    concept.concept_name AS measure_type  
                FROM {dataset_project}.{dataset}.measurement AS measure
                INNER JOIN {dataset_project}.{dataset}.concept AS concept
                ON measure.measurement_concept_id = concept.concept_id
                WHERE concept.concept_id=3027801 /*Oxygen [Partial pressure] in Arterial  blood*/ 
            ),
            glasgow_coma_scale_from_measurement AS (
                SELECT 
                    measure.person_id, 
                    measure.measurement_DATETIME, 
                    measure.value_as_number, 
                    concept.concept_name AS measure_type 
                FROM {dataset_project}.{dataset}.measurement AS measure
                INNER JOIN {dataset_project}.{dataset}.concept AS concept
                ON measure.measurement_concept_id = concept.concept_id
                where measure.measurement_concept_id = 3032652 and measure.value_as_number >= 3 and measure.value_as_number <= 15
            ),
            mean_arterial_pressure_from_measurement AS (
                SELECT 
                    measure.person_id, 
                    measure.measurement_DATETIME, 
                    measure.value_as_number, 
                    concept.concept_name AS measure_type 
                FROM {dataset_project}.{dataset}.measurement AS measure
                INNER JOIN {dataset_project}.{dataset}.concept AS concept
                ON measure.measurement_concept_id = concept.concept_id
                where measure.measurement_concept_id = 3027598
            ),
            mech_vent_from_flowsheet AS (
                SELECT 
                    person_id, 
                    observation_datetime, 
                    UPPER(display_name) as row_disp_name, 
                    meas_value, 
                    unit_value as units
                FROM {dataset_project}.{rs_dataset}.meas_vals_json
                where (upper(display_name) = 'VENT MODE' or upper(display_name) = 'VENTILATION MODE') and
                (meas_value <> 'STANDBY' and meas_value <> 'MONITOR' and meas_value is not null)   
            ),
            spO2_from_flowsheet AS (
                SELECT 
                    person_id, 
                    observation_datetime, 
                    UPPER(display_name) as row_disp_name, 
                    SAFE_CAST(meas_value as float64) as meas_value, 
                    unit_value as units
                FROM {dataset_project}.{rs_dataset}.meas_vals_json
                where ((upper(display_name) = 'OXYGEN SATURATION' AND upper(source_display_name) = 'SPO2') 
                or (upper(display_name) like 'SPO2 - %' and upper(source_display_name) = 'DEVICES TESTING TEMPLATE'))
                and meas_value is not null
            ),
            fiO2_vals as (
                SELECT 
                    person_id, 
                    observation_datetime, 
                    UPPER(display_name) as row_disp_name, 
                    SAFE_CAST(meas_value as float64) as meas_value, 
                    unit_value as units
                FROM {dataset_project}.{rs_dataset}.meas_vals_json
                where (upper(display_name) = 'FIO2 (%)' or (upper(display_name) = 'FIO2 %' and upper(source_display_name) = 'RN CLINICAL SCREENING'))
                      and meas_value is not null
            ),
            fiO2_from_flowsheet AS (
                select 
                    person_id, 
                    observation_datetime, 
                    row_disp_name,
                    case 
                        when meas_value > 1 then meas_value / 100
                        else meas_value
                    end as fiO2,
                    units
                from fiO2_vals
            ),
            urine_from_measurement AS (
                SELECT 
                    measure.person_id, 
                    measure.measurement_DATETIME,
                    measure.value_as_number, 
                    concept.concept_name AS measure_type 
                FROM {dataset_project}.{dataset}.measurement measure
                INNER JOIN {dataset_project}.{dataset}.concept AS concept
                ON measure.measurement_concept_id = concept.concept_id
                where measurement_concept_id = 45876241 
            ),
            # Assess as great as 48 hours before and up to 24 hours after index date
            platelet_window AS (
                SELECT 
                    susp_inf_rollup.*, 
                    platelet.measurement_DATETIME AS platelet_date, 
                    platelet.value_as_number,
                    datetime_diff(platelet.measurement_DATETIME, index_date, DAY) as days_plat_index
                FROM {suspected_infection} AS susp_inf_rollup
                LEFT JOIN platelet_from_measurement AS platelet USING (person_id)
                WHERE
                    CAST(index_date AS DATE) >= CAST(DATETIME_SUB(measurement_DATETIME, INTERVAL 2 DAY) AS DATE) AND
                    CAST(index_date AS DATE) <= CAST(DATETIME_ADD(measurement_DATETIME, INTERVAL 1 DAY) AS DATE) AND
                    value_as_number IS NOT NULL
            ),
            platelet_rollup AS (
                SELECT 
                    person_id, 
                    admit_date, 
                    MIN(value_as_number) as min_platelet
                FROM platelet_window 
                GROUP BY person_id, admit_date
            ),
            bilirubin_window AS (
                SELECT 
                    susp_inf_rollup.*, 
                    bilirubin.measurement_DATETIME AS bilirubin_date, 
                    bilirubin.value_as_number,
                    datetime_diff(bilirubin.measurement_DATETIME, index_date, DAY) as days_bili_index
                FROM {suspected_infection} AS susp_inf_rollup
                LEFT JOIN bilirubin_from_measurement AS bilirubin
                USING (person_id)
                WHERE
                    CAST(index_date AS DATE) >= CAST(DATETIME_SUB(measurement_DATETIME, INTERVAL 2 DAY) AS DATE) AND
                    CAST(index_date AS DATE) <= CAST(DATETIME_ADD(measurement_DATETIME, INTERVAL 1 DAY) AS DATE) AND
                    value_as_number > 0 AND value_as_number IS NOT NULL
            ),
            bilirubin_rollup AS (
                SELECT 
                    person_id, 
                    admit_date, 
                    MAX(value_as_number) as max_bilirubin
                FROM bilirubin_window 
                GROUP BY person_id, admit_date
            ),
            vasopressor_window AS (
                SELECT 
                    susp_inf_rollup.person_id, 
                    susp_inf_rollup.admit_date, 
                    susp_inf_rollup.index_date,
                    vasopressor.drug_exposure_start_DATETIME, 
                    vasopressor.drug_exposure_end_DATETIME,
                    datetime_diff(vasopressor.drug_exposure_start_DATETIME, index_date, DAY) as days_index_vasostart,
                    datetime_diff(vasopressor.drug_exposure_end_DATETIME, index_date, DAY) as days_index_vasoend,
                    (datetime_diff(vasopressor.drug_exposure_end_DATETIME, vasopressor.drug_exposure_start_DATETIME, DAY) +1 ) as days_vasopressor
                FROM {suspected_infection} AS susp_inf_rollup
                LEFT JOIN ionotrope_from_drug_exposure_with_name AS vasopressor
                USING (person_id)
                WHERE
                    CAST(index_date AS DATE) BETWEEN CAST(drug_exposure_start_DATETIME AS DATE) AND CAST(drug_exposure_end_DATETIME AS DATE) OR
                    CAST(DATETIME_ADD(index_date, INTERVAL 1 DAY) AS DATE) BETWEEN CAST(drug_exposure_start_DATETIME AS DATE) AND CAST (drug_exposure_end_DATETIME AS DATE) OR
                    CAST (DATETIME_SUB(index_date, INTERVAL 1 DAY) AS DATE) BETWEEN CAST (drug_exposure_start_DATETIME AS DATE) AND CAST (drug_exposure_end_DATETIME AS DATE) OR
                    CAST (DATETIME_SUB(index_date, INTERVAL 2 DAY) AS DATE) BETWEEN CAST (drug_exposure_start_DATETIME AS DATE) AND CAST (drug_exposure_end_DATETIME AS DATE)
            ),
            vasopressor_rollup AS (
                SELECT 
                    person_id, 
                    admit_date, 
                    MAX(datetime_diff(vasopressor.drug_exposure_end_DATETIME, 
                    vasopressor.drug_exposure_start_DATETIME, DAY) +1) as max_vaso_days
                FROM vasopressor_window as vasopressor 
                GROUP BY person_id, admit_date
            ),
            creatinine_window AS (
                SELECT 
                    susp_inf_rollup.*, 
                    creatinine.measurement_DATETIME AS creatinine_date, 
                    creatinine.value_as_number,
                    datetime_diff(creatinine.measurement_DATETIME, index_date, DAY) as days_crea_index
                FROM {suspected_infection} AS susp_inf_rollup
                LEFT JOIN creatinine_from_measurement AS creatinine
                USING (person_id)
                WHERE
                    CAST(index_date AS DATE) >= CAST(DATETIME_SUB(measurement_DATETIME, INTERVAL 2 DAY) AS DATE) AND
                    CAST(index_date AS DATE) <= CAST(DATETIME_ADD(measurement_DATETIME, INTERVAL 1 DAY) AS DATE) AND
                    value_as_number IS NOT NULL
            ),
            creatinine_rollup AS (
                SELECT 
                    person_id, 
                    admit_date,
                    MAX(value_as_number) as max_creatinine
                FROM creatinine_window 
                GROUP BY person_id, admit_date
            ),
            lactate_window AS (
                SELECT 
                    susp_inf_rollup.*, 
                    lactate.measurement_DATETIME AS lactate_date, 
                    lactate.value_as_number,
                    datetime_diff(lactate.measurement_DATETIME, index_date, DAY) as days_lact_index
                FROM {suspected_infection} AS susp_inf_rollup
                LEFT JOIN lactate_from_measurement AS lactate
                USING (person_id)
                WHERE
                    CAST(index_date AS DATE) >= CAST(DATETIME_SUB(measurement_DATETIME, INTERVAL 2 DAY) AS DATE) AND
                    CAST(index_date AS DATE) <= CAST(DATETIME_ADD(measurement_DATETIME, INTERVAL 1 DAY) AS DATE) AND
                    value_as_number IS NOT NULL
            ),
            lactate_rollup AS (
                SELECT 
                    person_id, 
                    admit_date, 
                    MAX(value_as_number) as max_lactate
                FROM lactate_window 
                GROUP BY person_id, admit_date
            ),
            paO2_window AS (
                SELECT 
                    susp_inf_rollup.*, 
                    paO2.measurement_DATETIME AS paO2_datetime, 
                    paO2.value_as_number as paO2,
                    datetime_diff(paO2.measurement_DATETIME, index_date, DAY) as days_paO2_index
                FROM {suspected_infection} AS susp_inf_rollup
                LEFT JOIN paO2_from_measurement AS paO2
                USING (person_id)
                WHERE
                    CAST(index_date AS DATE) >= CAST(DATETIME_SUB(measurement_DATETIME, INTERVAL 2 DAY) AS DATE) AND
                    CAST(index_date AS DATE) <= CAST(DATETIME_ADD(measurement_DATETIME, INTERVAL 1 DAY) AS DATE) AND
                    value_as_number IS NOT NULL
            ),
            fiO2_window AS ( 
                SELECT 
                    susp_inf_rollup.person_id, 
                    min_bc, 
                    min_systemic_abx, 
                    susp_inf_rollup.admit_date, 
                    index_date, 
                    CAST(observation_datetime AS DATETIME) AS fiO2_datetime, 
                    fiO2, 
                    datetime_diff(CAST(observation_datetime AS DATETIME), index_date, DAY) as days_fiO2_index
                FROM {suspected_infection} AS susp_inf_rollup
                LEFT JOIN fiO2_from_flowsheet as flowsheet
                ON susp_inf_rollup.person_id = flowsheet.person_id
                WHERE
                CAST(index_date AS DATE) >= CAST(DATETIME_SUB(CAST(observation_datetime AS DATETIME), INTERVAL 2 DAY) AS DATE) AND
                CAST(index_date As DATE) <= CAST(DATETIME_ADD(CAST(observation_datetime AS DATETIME), INTERVAL 1 DAY) AS DATE) AND
                fiO2 >=0.21 AND fiO2 <=1.0
            ),
            paO2_fiO2_window AS (
                SELECT 
                    paO2_window.person_id, 
                    paO2_window.admit_date, 
                    paO2_window.index_date, 
                    fiO2, 
                    fiO2_datetime, 
                    paO2, 
                    paO2_datetime,  
                    paO2/(NULLIF(fiO2, 0)) AS paO2fiO2_ratio, 
                    datetime_diff(paO2_datetime, fiO2_datetime, MINUTE) as minutes_fiO2_paO2
                FROM fiO2_window AS fiO2_window
                INNER JOIN paO2_window AS paO2_window
                USING (person_id, index_date)
                WHERE CAST(fiO2_datetime AS DATETIME)<= paO2_datetime 
            ),
            paO2_fiO2_initial_rollup AS (
                SELECT 
                    person_id, 
                    admit_date, 
                    paO2_datetime, 
                    MIN(minutes_fiO2_paO2) As minutes_fiO2_paO2 
                FROM paO2_fiO2_window 
                GROUP BY person_id, admit_date, paO2_datetime
            ),
            paO2_fiO2_initial_rollup_join AS (
                SELECT 
                    initial_rollup.person_id, 
                    initial_rollup.admit_date, 
                    initial_rollup.paO2_datetime, 
                    initial_rollup.minutes_fiO2_paO2,
                    index_date, 
                    fiO2, fiO2_datetime, paO2, paO2fiO2_ratio
                FROM paO2_fiO2_initial_rollup AS initial_rollup
                LEFT JOIN paO2_fiO2_window AS combined_window
                USING (person_id, paO2_datetime, minutes_fiO2_paO2)
            ),
            paO2_fiO2_rollup AS (
                SELECT 
                    person_id, 
                    CAST(admit_date AS DATE) as admit_date, 
                    MIN(paO2fiO2_ratio) as min_paO2fiO2_ratio
                FROM paO2_fiO2_initial_rollup_join 
                WHERE minutes_fiO2_paO2 <= 24*60
                GROUP BY person_id, admit_date
            ),
            spO2_window AS ( 
                SELECT 
                    susp_inf_rollup.person_id, 
                    min_bc, 
                    min_systemic_abx, 
                    susp_inf_rollup.admit_date, 
                    index_date, 
                    CAST(observation_datetime AS DATETIME) AS spO2_datetime, 
                    meas_value as spO2, 
                    datetime_diff(CAST(observation_datetime AS DATETIME), index_date, DAY) as days_spO2_index
                FROM {suspected_infection} AS susp_inf_rollup
                LEFT JOIN spO2_from_flowsheet as flowsheet
                ON susp_inf_rollup.person_id = flowsheet.person_id
                WHERE
                CAST(index_date AS DATE) >= CAST(DATETIME_SUB(CAST(observation_datetime AS DATETIME), INTERVAL 2 DAY) AS DATE) AND
                CAST(index_date As DATE) <= CAST(DATETIME_ADD(CAST(observation_datetime AS DATETIME), INTERVAL 1 DAY) AS DATE) AND
                meas_value >0 AND meas_value <=100
            ),
            spO2_fiO2_window AS (
                SELECT 
                    spO2_window.person_id, 
                    spO2_window.admit_date, 
                    spO2_window.index_date, 
                    fiO2, 
                    fiO2_datetime, 
                    spO2, 
                    spO2_datetime,  
                    spO2/(NULLIF(fiO2, 0)) AS spO2fiO2_ratio, 
                    datetime_diff(spO2_datetime, fiO2_datetime, MINUTE) as minutes_fiO2_spO2
                FROM fiO2_window AS fiO2_window
                INNER JOIN spO2_window AS spO2_window
                USING (person_id, index_date)
                WHERE CAST(fiO2_datetime AS DATETIME)<= spO2_datetime 
            ),
            spO2_fiO2_initial_rollup AS (
                SELECT 
                    person_id, 
                    admit_date, 
                    spO2_datetime, 
                    MIN(minutes_fiO2_spO2) As minutes_fiO2_spO2 
                FROM spO2_fiO2_window 
                GROUP BY person_id, admit_date, spO2_datetime
            ),
            spO2_fiO2_initial_rollup_join AS (
                SELECT 
                    initial_rollup.person_id, 
                    initial_rollup.admit_date, 
                    initial_rollup.spO2_datetime, 
                    initial_rollup.minutes_fiO2_spO2,
                    index_date, 
                    fiO2, 
                    fiO2_datetime, 
                    spO2, 
                    spO2fiO2_ratio
                FROM spO2_fiO2_initial_rollup AS initial_rollup
                LEFT JOIN spO2_fiO2_window AS combined_window
                USING (person_id, spO2_datetime, minutes_fiO2_spO2)
            ),
            spO2_fiO2_rollup AS (
                SELECT
                    person_id, 
                    CAST(admit_date AS DATE) as admit_date, 
                    MIN(spO2fiO2_ratio) as min_spO2fiO2_ratio
                FROM spO2_fiO2_initial_rollup_join 
                WHERE minutes_fiO2_spO2 <= 24*60
                GROUP BY person_id, admit_date
            ),
            glasgow_coma_scale_window AS (
                SELECT 
                    susp_inf_rollup.*, 
                    gcs.measurement_DATETIME AS gcs_datetime, 
                    gcs.value_as_number AS glasgow_coma_scale,
                    datetime_diff(gcs.measurement_DATETIME, index_date, DAY) as days_gcs_index
                FROM {suspected_infection} AS susp_inf_rollup
                LEFT JOIN glasgow_coma_scale_from_measurement as gcs
                ON susp_inf_rollup.person_id =  gcs.person_id
                WHERE
                CAST(index_date AS DATE) >= CAST(DATETIME_SUB(measurement_DATETIME, INTERVAL 2 DAY) AS DATE) AND
                CAST(index_date AS DATE) <= CAST(DATETIME_ADD(measurement_DATETIME, INTERVAL 1 DAY) AS DATE)
            ),
            glasgow_coma_scale_rollup AS (
                SELECT 
                    person_id, 
                    admit_date, 
                    MIN(glasgow_coma_scale) as min_gcs
                FROM glasgow_coma_scale_window 
                GROUP BY person_id, admit_date
            ),
            mean_arterial_pressure_window AS (
                SELECT 
                    susp_inf_rollup.*, 
                    mapm.measurement_DATETIME AS map_datetime,
                    mapm.value_as_number AS map,
                    datetime_diff(mapm.measurement_DATETIME, index_date, DAY) as days_map_index
                FROM {suspected_infection} AS susp_inf_rollup
                LEFT JOIN mean_arterial_pressure_from_measurement as mapm
                    ON susp_inf_rollup.person_id = mapm.person_id
                WHERE
                    CAST(index_date AS DATE) >= CAST(DATETIME_SUB(mapm.measurement_DATETIME, INTERVAL 2 DAY) AS DATE) AND
                    CAST(index_date AS DATE) <= CAST(DATETIME_ADD(mapm.measurement_DATETIME, INTERVAL 1 DAY) AS DATE) AND
                    mapm.value_as_number IS NOT NULL
            ),
            mean_arterial_pressure_rollup AS (
                SELECT 
                    person_id, 
                    admit_date, 
                    MIN(map) as min_map
                FROM mean_arterial_pressure_window 
                WHERE map >=10
                GROUP BY person_id, admit_date
            ),
            mech_vent_window AS (
            SELECT 
                susp_inf_rollup.*, 
                mech_vent.observation_datetime AS mech_vent_datetime, 
                mech_vent.meas_value as vent_mode,
                datetime_diff(mech_vent.observation_datetime, index_date, DAY) as days_mech_vent_index
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN mech_vent_from_flowsheet AS mech_vent USING (person_id)
            WHERE
                CAST(index_date AS DATE) >= CAST(DATETIME_SUB(observation_datetime, INTERVAL 2 DAY) AS DATE) AND
                CAST(index_date AS DATE) <= CAST(DATETIME_ADD(observation_datetime, INTERVAL 1 DAY) AS DATE)
            ORDER BY  person_id, admit_date, index_date, observation_datetime 
            ),
            mech_vent_rollup AS (
                SELECT 
                    person_id, 
                    admit_date, 
                    COUNT(vent_mode) as count_vent_mode
                FROM mech_vent_window 
                GROUP BY person_id, admit_date
            ),
            urine_output_window AS (
                SELECT 
                    susp_inf_rollup.*, 
                    urine.measurement_DATETIME AS urine_datetime, 
                    urine.value_as_number AS urine_volume,
                    datetime_diff(measurement_DATETIME, index_date, DAY) as days_urine_index
                FROM {suspected_infection} AS susp_inf_rollup
                LEFT JOIN urine_from_measurement as urine
                    ON susp_inf_rollup.person_id = urine.person_id 
                WHERE
                    CAST(index_date AS DATE) >= CAST(DATETIME_SUB(measurement_DATETIME, INTERVAL 2 DAY) AS DATE) AND
                    CAST(index_date As DATE) <= CAST(DATETIME_ADD(measurement_DATETIME, INTERVAL 1 DAY) AS DATE) AND
                    value_as_number IS NOT NULL AND value_as_number >=0
                ORDER BY person_id, admit_date, measurement_DATETIME
            ),
            admit_time AS (
                SELECT 
                    person_id, 
                    MIN(observation_datetime) AS ext_urine_datetime,
                    EXTRACT(HOUR FROM MIN(observation_datetime)) AS hour, 
                    (24-EXTRACT(HOUR FROM MIN(observation_datetime))) AS adjust_hours
                FROM urine_output_window AS urine
                LEFT JOIN som-nero-nigam-starr.jlemmon_explore.meas_vals_json AS flowsheets_orig  USING (person_id)
                WHERE CAST(admit_date AS DATE) = CAST(observation_datetime AS DATE)  
                AND observation_datetime <> DATETIME_TRUNC(observation_datetime, DAY)
                GROUP BY person_id
            ),
            discharge_time AS (
                SELECT 
                    person_id, 
                    MAX(observation_datetime) AS ext_urine_datetime,
                    EXTRACT(HOUR FROM MAX(observation_datetime)) AS adjust_hours
                FROM urine_output_window AS urine
                LEFT JOIN som-nero-nigam-starr.jlemmon_explore.meas_vals_json AS flowsheets_orig  USING (person_id)
                WHERE CAST(admit_date AS DATE) = CAST(observation_datetime AS DATE)  
                AND observation_datetime <> DATETIME_TRUNC(observation_datetime, DAY)
                GROUP BY person_id
            ),
            urine_output_initial_rollup AS (
                (
                    SELECT 
                        person_id, 
                        admit_date, 
                        discharge_date, 
                        CAST(urine_datetime AS DATE) AS urine_date, 
                        SUM(urine_volume) as urine_daily_output_orig, 
                        SUM(urine_volume) as urine_daily_output_adj,
                        ext_urine_datetime, 
                        adjust_hours
                    FROM urine_output_window 
                    LEFT JOIN admit_time USING (person_id)
                    WHERE CAST(urine_datetime AS DATE) <> CAST(admit_date AS DATE) AND CAST(urine_datetime AS DATE) <> CAST (discharge_date AS DATE)
                    GROUP BY person_id, admit_date, discharge_date, CAST (urine_datetime AS DATE), ext_urine_datetime, adjust_hours 
                )
                UNION ALL
                (
                    SELECT 
                        person_id, 
                        admit_date, 
                        discharge_date, 
                        CAST(urine_datetime AS DATE) AS urine_date, 
                        SUM(urine_volume) as urine_daily_output_orig, 
                        (SUM(urine_volume))*24/adjust_hours as urine_daily_output_adj,
                        ext_urine_datetime, 
                        adjust_hours
                    FROM urine_output_window 
                    LEFT JOIN admit_time USING (person_id)
                    WHERE CAST(urine_datetime AS DATE) = CAST(admit_date AS DATE) 
                    GROUP BY person_id, admit_date, discharge_date, CAST(urine_datetime AS DATE), ext_urine_datetime, adjust_hours
                )
                UNION ALL
                (
                    SELECT 
                        person_id, 
                        admit_date, 
                        discharge_date, 
                        CAST(urine_datetime AS DATE) AS urine_date, 
                        SUM(urine_volume) as urine_daily_output_orig, (SUM(urine_volume))*24/adjust_hours as urine_daily_output_adj,
                        ext_urine_datetime, 
                        adjust_hours
                    FROM urine_output_window 
                    LEFT JOIN discharge_time USING (person_id)
                    WHERE CAST(urine_datetime AS DATE) = CAST(discharge_date AS DATE) AND adjust_hours <> 0 
                    GROUP BY person_id, admit_date, discharge_date, CAST(urine_datetime AS DATE), ext_urine_datetime, adjust_hours
                )
                UNION ALL # THIS LAST BIT DEALS WITH DISCHARGE AT 0:00:00 HOURS
                (
                    SELECT 
                        person_id, 
                        admit_date, 
                        discharge_date, 
                        CAST(urine_datetime AS DATE) AS urine_date, 
                        SUM(urine_volume) as urine_daily_output_orig, (SUM(urine_volume))*24 as urine_daily_output_adj,
                        ext_urine_datetime, 
                        adjust_hours
                    FROM urine_output_window 
                    LEFT JOIN discharge_time USING (person_id)
                    WHERE CAST(urine_datetime AS DATE) = CAST(discharge_date AS DATE) AND adjust_hours = 0 
                    GROUP BY person_id, admit_date, discharge_date, CAST(urine_datetime AS DATE), ext_urine_datetime, adjust_hours
                )
            ),
            urine_output_rollup AS (
                SELECT 
                    person_id, 
                    admit_date, 
                    Min(urine_daily_output_adj) as min_urine_daily
                FROM urine_output_initial_rollup 
                GROUP BY person_id, admit_date
            )
            # Begin sepsis and shock cohort
            SELECT 
                susp_inf_rollup.person_id, 
                CAST(susp_inf_rollup.admit_date AS DATE) AS admit_date, 
                CAST(discharge_date AS DATE) AS discharge_date,
                min_bc, 
                min_systemic_abx, 
                index_date, 
                min_platelet, 
                CASE 
                    WHEN min_platelet IS NULL THEN 0 
                    WHEN min_platelet <20 THEN 4 
                    WHEN min_platelet < 50 THEN 3
                    WHEN min_platelet < 100 THEN 2 
                    WHEN min_platelet < 150 THEN 1 
                    ELSE 0 
                END plat_SOFA,
                max_bilirubin, 
                CASE 
                    WHEN max_bilirubin IS NULL THEN 0 
                    WHEN max_bilirubin >= 12 THEN 4 
                    WHEN max_bilirubin >= 6 THEN 3
                    WHEN max_bilirubin >= 2 THEN 2 
                    WHEN max_bilirubin >= 1.2 THEN 1 
                    ELSE 0
                END bili_SOFA,
                max_creatinine, 
                CASE 
                    WHEN max_creatinine IS NULL THEN 0 
                    WHEN max_creatinine >= 5 THEN 4 
                    WHEN max_creatinine >= 3.5 THEN 3
                    WHEN max_creatinine >= 2 THEN 2 
                    WHEN max_creatinine >= 1.2 THEN 1 
                    ELSE 0 
                END crea_SOFA,
                max_vaso_days, 
                min_map, 
                CASE 
                    WHEN max_vaso_days IS NOT NULL THEN 2
                    WHEN min_map <70 THEN 1 
                    ELSE 0 
                END cv_SOFA,
                min_paO2fiO2_ratio,
                min_spO2fiO2_ratio,
                count_vent_mode, 
                CASE 
                    WHEN (count_vent_mode IS NOT NULL AND min_paO2fiO2_ratio < 100) THEN 4
                    WHEN count_vent_mode IS NOT NULL THEN 3 
                    WHEN min_paO2fiO2_ratio < 300 THEN 2 
                    WHEN min_paO2fiO2_ratio < 400 THEN 1
                    ELSE 0 
                END resp_SOFA,
                min_gcs, 
                CASE 
                    WHEN min_gcs IS NULL THEN 0 
                    WHEN min_gcs < 6 THEN 4 
                    WHEN min_gcs < 10 THEN 3
                    WHEN min_gcs < 13 THEN 2 
                    WHEN min_gcs < 15 THEN 1 
                    ELSE 0 
                END gcs_SOFA,
                min_urine_daily, 
                CASE 
                    WHEN min_urine_daily IS NULL THEN 0 
                    WHEN min_urine_daily < 200 THEN 4 
                    WHEN min_urine_daily < 500 THEN 3 
                    ELSE 0 
                END urine_SOFA,
                CASE 
                    WHEN max_vaso_days IS NOT NULL THEN 1 
                    ELSE 0 
                END vaso_shock,
                CASE 
                    WHEN min_map < 65 THEN 1 
                    ELSE 0 
                END map_shock,
                max_lactate, 
                CASE 
                    WHEN max_lactate > 2 THEN 1 
                    ELSE 0 
                END lact_shock
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN platelet_rollup USING (person_id, admit_date)
            LEFT JOIN bilirubin_rollup USING (person_id, admit_date)
            LEFT JOIN creatinine_rollup USING (person_id, admit_date)
            LEFT JOIN vasopressor_rollup USING (person_id, admit_date)
            LEFT JOIN mean_arterial_pressure_rollup USING (person_id, admit_date)
            LEFT JOIN paO2_fiO2_rollup USING (person_id, admit_date)
            LEFT JOIN mech_vent_rollup USING (person_id, admit_date)
            LEFT JOIN glasgow_coma_scale_rollup USING (person_id, admit_date)
            LEFT JOIN urine_output_rollup USING (person_id, admit_date)
            LEFT JOIN lactate_rollup USING (person_id, admit_date)
            LEFT JOIN spO2_fiO2_rollup USING (person_id, admit_date)
            ORDER BY person_id, CAST(admit_date AS DATE)
        """

        if not format_query:
            return query
        else:
            return query.format_map(self.config_dict)

    def get_sepsis_cohort_prior(self, format_query=True):
        query = """
        WITH 
        platelet_from_measurement AS (
            SELECT 
                measure.person_id, 
                measure.measurement_DATETIME, 
                measure.value_as_number, 
                concept.concept_name AS measure_type  
            FROM {dataset_project}.{dataset}.measurement AS measure
            INNER JOIN {dataset_project}.{dataset}.concept AS concept
            ON measure.measurement_concept_id = concept.concept_id
            WHERE concept.concept_id in(
                SELECT
                    c.concept_id
                FROM {dataset_project}.{dataset}.concept c
                WHERE c.concept_id in (37037425,40654106)
                UNION DISTINCT
                SELECT
                    c.concept_id
                FROM {dataset_project}.{dataset}.concept c
                INNER JOIN {dataset_project}.{dataset}.concept_ancestor ca
                ON c.concept_id = ca.descendant_concept_id
                AND ca.ancestor_concept_id in (37037425,40654106)
                AND c.invalid_reason is null
            )
        ),
        bilirubin_from_measurement AS (
            SELECT 
                measure.person_id, 
                measure.measurement_DATETIME, 
                measure.value_as_number, 
                concept.concept_name AS measure_type  
            FROM {dataset_project}.{dataset}.measurement AS measure
            INNER JOIN {dataset_project}.{dataset}.concept AS concept
            ON measure.measurement_concept_id = concept.concept_id
            WHERE concept.concept_id in (
                SELECT
                    c.concept_id
                FROM {dataset_project}.{dataset}.concept c
                WHERE c.concept_id in (3024128, 4230543)
                UNION DISTINCT
                SELECT
                    c.concept_id
                FROM {dataset_project}.{dataset}.concept c
                INNER JOIN {dataset_project}.{dataset}.concept_ancestor ca
                ON c.concept_id = ca.descendant_concept_id
                AND ca.ancestor_concept_id in (3024128, 4230543)
                AND c.invalid_reason is null
            )
        ),
        ionotrope_list AS (
            SELECT 
                descendant_concept_id AS concept_id
            FROM {dataset_project}.{dataset}.concept_ancestor
            WHERE ancestor_concept_id IN (21600284, 21600287, 21600303, 21600283, 21600308)
        ),
        ionotrope_from_drug_exposure_via_ancestor AS ( 
            SELECT *
            FROM {dataset_project}.{dataset}.drug_exposure AS drug
            WHERE drug.drug_concept_id IN (
                SELECT concept_id
                FROM ionotrope_list
            )
        ),
        ionotrope_from_drug_exposure_with_name AS (
            SELECT 
                ionotrope.*, 
                concept.concept_name AS ionotrope_type  
            FROM ionotrope_from_drug_exposure_via_ancestor AS ionotrope
            INNER JOIN {dataset_project}.{dataset}.concept AS concept
            ON ionotrope.drug_concept_id = concept.concept_id
        ),
        creatinine_from_measurement AS (
            SELECT 
                measure.person_id, 
                measure.measurement_DATETIME, 
                measure.value_as_number, 
                concept.concept_name AS measure_type  
            FROM {dataset_project}.{dataset}.measurement AS measure
            INNER JOIN {dataset_project}.{dataset}.concept AS concept
            ON measure.measurement_concept_id = concept.concept_id
            WHERE concept.concept_id in (
                SELECT
                    c.concept_id
                FROM {dataset_project}.{dataset}.concept c
                WHERE c.concept_id in (37029387,4013964,2212294,3051825)
                UNION DISTINCT
                SELECT
                    c.concept_id
                FROM {dataset_project}.{dataset}.concept c
                INNER JOIN {dataset_project}.{dataset}.concept_ancestor ca
                ON c.concept_id = ca.descendant_concept_id
                AND ca.ancestor_concept_id in (37029387,4013964,2212294,3051825)
                AND c.invalid_reason is null

            )
        ),
        lactate_from_measurement AS (
            SELECT 
                measure.person_id, 
                measure.measurement_DATETIME, 
                measure.value_as_number, 
                concept.concept_name AS measure_type  
            FROM {dataset_project}.{dataset}.measurement AS measure
            INNER JOIN {dataset_project}.{dataset}.concept AS concept
            ON measure.measurement_concept_id = concept.concept_id
            WHERE concept.concept_id = 3047181
            UNION ALL
            SELECT 
                measure.person_id, 
                measure.measurement_DATETIME, 
                measure.value_as_number/9.0,
                concept.concept_name AS measure_type  
            FROM {dataset_project}.{dataset}.measurement AS measure /* Normalize the mg/dL measurement to mmol/L */
            INNER JOIN {dataset_project}.{dataset}.concept AS concept
            ON measure.measurement_concept_id = concept.concept_id
            WHERE concept.concept_id = 3020138
        ),
        paO2_from_measurement AS (
            SELECT 
                measure.person_id, 
                measure.measurement_DATETIME, 
                measure.value_as_number, 
                concept.concept_name AS measure_type  
            FROM {dataset_project}.{dataset}.measurement AS measure
            INNER JOIN {dataset_project}.{dataset}.concept AS concept
            ON measure.measurement_concept_id = concept.concept_id
            WHERE concept.concept_id=3027801 /*Oxygen [Partial pressure] in Arterial  blood*/ 
        ),
        glasgow_coma_scale_from_measurement AS (
            SELECT 
                measure.person_id, 
                measure.measurement_DATETIME, 
                measure.value_as_number,
                concept.concept_name AS measure_type 
            FROM {dataset_project}.{dataset}.measurement AS measure
            INNER JOIN {dataset_project}.{dataset}.concept AS concept
            ON measure.measurement_concept_id = concept.concept_id
            where measure.measurement_concept_id = 3032652 and measure.value_as_number >= 3 and measure.value_as_number <= 15
        ),
        mean_arterial_pressure_from_measurement AS (
            SELECT 
                measure.person_id, 
                measure.measurement_DATETIME, 
                measure.value_as_number, 
                concept.concept_name AS measure_type 
            FROM {dataset_project}.{dataset}.measurement AS measure
            INNER JOIN {dataset_project}.{dataset}.concept AS concept
            ON measure.measurement_concept_id = concept.concept_id
            where measure.measurement_concept_id = 3027598
        ),
        mech_vent_from_flowsheet AS (
            SELECT 
                person_id, 
                observation_datetime, 
                UPPER(display_name) as row_disp_name,
                meas_value, 
                unit_value as units
            FROM {dataset_project}.{rs_dataset}.meas_vals_json
            where (upper(display_name) = 'VENT MODE' or upper(display_name) = 'VENTILATION MODE') and
            (meas_value <> 'STANDBY' and meas_value <> 'MONITOR' and meas_value is not null)   
        ),
        spO2_from_flowsheet AS (
            SELECT 
                person_id, 
                observation_datetime, 
                UPPER(display_name) as row_disp_name, 
                SAFE_CAST(meas_value as float64) as meas_value, 
                unit_value as units
            FROM {dataset_project}.{rs_dataset}.meas_vals_json
            where ((upper(display_name) = 'OXYGEN SATURATION' AND upper(source_display_name) = 'SPO2') 
            or (upper(display_name) like 'SPO2 - %' and upper(source_display_name) = 'DEVICES TESTING TEMPLATE'))
            and meas_value is not null
        ),
        fiO2_vals as (
            SELECT 
                person_id, 
                observation_datetime, 
                UPPER(display_name) as row_disp_name, 
                SAFE_CAST(meas_value as float64) as meas_value, 
                unit_value as units
            FROM {dataset_project}.{rs_dataset}.meas_vals_json 
            where (upper(display_name) = 'FIO2 (%)' or (upper(display_name) = 'FIO2 %' and upper(source_display_name) = 'RN CLINICAL SCREENING'))
            and meas_value is not null
        ),
        fiO2_from_flowsheet AS (
            select 
                person_id, 
                observation_datetime, 
                row_disp_name,
                case 
                    when meas_value > 1 then meas_value / 100
                    else meas_value
                end as fiO2,
                units
            from fiO2_vals
        ),
        urine_from_measurement AS (
            SELECT 
                measure.person_id, 
                measure.measurement_DATETIME, 
                measure.value_as_number, 
                concept.concept_name AS measure_type 
            FROM {dataset_project}.{dataset}.measurement measure
            INNER JOIN {dataset_project}.{dataset}.concept AS concept
            ON measure.measurement_concept_id = concept.concept_id
            where measurement_concept_id = 45876241 
        ),
        # Assess 10 days to 3 days prior to index date
        platelet_window AS (
            SELECT 
                susp_inf_rollup.*, 
                platelet.measurement_DATETIME AS platelet_date, 
                platelet.value_as_number,
                datetime_diff(platelet.measurement_DATETIME, index_date, DAY) as days_plat_index
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN platelet_from_measurement AS platelet
            USING (person_id)
            WHERE
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 2 DAY) > CAST(measurement_DATETIME AS DATE) AND
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 10 DAY) <= CAST(measurement_DATETIME AS DATE) AND
                value_as_number IS NOT NULL
        ),
        platelet_rollup AS (
            SELECT 
                person_id, 
                admit_date, 
                MIN(value_as_number) as min_platelet_prior
            FROM platelet_window 
            GROUP BY person_id, admit_date
        ),
        bilirubin_window AS (
            SELECT 
                susp_inf_rollup.*, 
                bilirubin.measurement_DATETIME AS bilirubin_date, 
                bilirubin.value_as_number,
                datetime_diff(bilirubin.measurement_DATETIME, index_date, DAY) as days_bili_index
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN bilirubin_from_measurement AS bilirubin
            USING (person_id)
            WHERE
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 2 DAY) > CAST(measurement_DATETIME AS DATE) AND
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 10 DAY) <= CAST(measurement_DATETIME AS DATE) AND
                value_as_number > 0 AND value_as_number IS NOT NULL
        ),
        bilirubin_rollup AS (
            SELECT 
                person_id, 
                admit_date, 
                MAX(value_as_number) as max_bilirubin_prior
            FROM bilirubin_window 
            GROUP BY person_id, admit_date
        ), 
        vasopressor_window AS (
            SELECT 
                susp_inf_rollup.person_id, 
                susp_inf_rollup.admit_date, 
                susp_inf_rollup.index_date,
                vasopressor.drug_exposure_start_DATETIME, 
                vasopressor.drug_exposure_end_DATETIME,
                datetime_diff(vasopressor.drug_exposure_start_DATETIME, index_date, DAY) as days_index_vasostart,
                datetime_diff(vasopressor.drug_exposure_end_DATETIME, index_date, DAY) as days_index_vasoend,
                (datetime_diff(vasopressor.drug_exposure_end_DATETIME, vasopressor.drug_exposure_start_DATETIME, DAY) + 1) as days_vasopressor
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN ionotrope_from_drug_exposure_with_name AS vasopressor
            USING (person_id)
            WHERE
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 3 DAY) BETWEEN CAST (drug_exposure_start_DATETIME AS DATE) AND CAST (drug_exposure_end_DATETIME AS DATE) OR
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 4 DAY) BETWEEN CAST (drug_exposure_start_DATETIME AS DATE) AND CAST (drug_exposure_end_DATETIME AS DATE) OR
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 5 DAY) BETWEEN CAST (drug_exposure_start_DATETIME AS DATE) AND CAST (drug_exposure_end_DATETIME AS DATE) OR
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 6 DAY) BETWEEN CAST (drug_exposure_start_DATETIME AS DATE) AND CAST (drug_exposure_end_DATETIME AS DATE) OR
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 7 DAY) BETWEEN CAST (drug_exposure_start_DATETIME AS DATE) AND CAST (drug_exposure_end_DATETIME AS DATE) OR
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 8 DAY) BETWEEN CAST (drug_exposure_start_DATETIME AS DATE) AND CAST (drug_exposure_end_DATETIME AS DATE) OR
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 9 DAY) BETWEEN CAST (drug_exposure_start_DATETIME AS DATE) AND CAST (drug_exposure_end_DATETIME AS DATE) OR
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 10 DAY) BETWEEN CAST (drug_exposure_start_DATETIME AS DATE) AND CAST (drug_exposure_end_DATETIME AS DATE)
        ),
        vasopressor_rollup AS (
            SELECT 
                person_id, 
                admit_date, 
                MAX(datetime_diff(vasopressor.drug_exposure_end_DATETIME, vasopressor.drug_exposure_start_DATETIME, DAY) + 1)
            as max_vaso_days_prior
            FROM vasopressor_window as vasopressor 
            GROUP BY person_id, admit_date
        ),
        creatinine_window AS (
            SELECT 
                susp_inf_rollup.*, 
                creatinine.measurement_DATETIME AS creatinine_date, 
                creatinine.value_as_number,
                datetime_diff(creatinine.measurement_DATETIME, index_date, DAY) as days_crea_index
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN creatinine_from_measurement AS creatinine
            USING (person_id)
            WHERE
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 2 DAY) > CAST(measurement_DATETIME AS DATE) AND
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 10 DAY) <= CAST(measurement_DATETIME AS DATE) AND
                value_as_number IS NOT NULL
        ),
        creatinine_rollup AS (
            SELECT 
                person_id, 
                admit_date, 
                MAX(value_as_number) as max_creatinine_prior
            FROM creatinine_window 
            GROUP BY person_id, admit_date
        ),
        lactate_window AS (
            SELECT 
                susp_inf_rollup.*, 
                lactate.measurement_DATETIME AS lactate_date, 
                lactate.value_as_number,
                datetime_diff(lactate.measurement_DATETIME, index_date, DAY) as days_lact_index
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN lactate_from_measurement AS lactate
            USING (person_id)
            WHERE
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 2 DAY) > CAST(measurement_DATETIME AS DATE) AND
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 10 DAY) <= CAST(measurement_DATETIME AS DATE) AND
                value_as_number IS NOT NULL
        ),
        lactate_rollup AS (
            SELECT 
                person_id, 
                admit_date, 
                MAX(value_as_number) as max_lactate_prior
            FROM lactate_window 
            GROUP BY person_id, admit_date
        ),
        paO2_window AS (
            SELECT 
                susp_inf_rollup.*, 
                paO2.measurement_DATETIME AS paO2_datetime, 
                paO2.value_as_number as paO2,
                datetime_diff(paO2.measurement_DATETIME, index_date, DAY) as days_paO2_index
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN paO2_from_measurement AS paO2
            USING (person_id)
            WHERE
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 2 DAY) > CAST(measurement_DATETIME AS DATE) AND
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 10 DAY) <= CAST(measurement_DATETIME AS DATE) AND
                value_as_number IS NOT NULL
        ),
        fiO2_window AS ( 
            SELECT 
                susp_inf_rollup.person_id, 
                min_bc, 
                min_systemic_abx, 
                susp_inf_rollup.admit_date, 
                index_date, 
                CAST(observation_datetime AS DATETIME) AS fiO2_datetime, 
                fiO2, 
                datetime_diff(CAST(observation_datetime AS DATETIME), index_date, DAY) as days_fiO2_index
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN fiO2_from_flowsheet as flowsheet
                ON susp_inf_rollup.person_id = flowsheet.person_id
            WHERE
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 2 DAY) > CAST(observation_datetime AS DATE) AND
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 10 DAY) <= CAST(observation_datetime AS DATE) AND
                fiO2 >=0.21 AND fiO2 <=1.0
        ),
        paO2_fiO2_window AS (
            SELECT 
                paO2_window.person_id, 
                paO2_window.admit_date, 
                paO2_window.index_date, 
                fiO2, 
                fiO2_datetime, 
                paO2, 
                paO2_datetime,  
                paO2/(NULLIF(fiO2, 0)) AS paO2fiO2_ratio, 
                datetime_diff(paO2_datetime, fiO2_datetime, MINUTE) as minutes_fiO2_paO2
            FROM fiO2_window AS fiO2_window
            INNER JOIN paO2_window AS paO2_window
            USING (person_id, index_date)
            WHERE CAST(fiO2_datetime AS DATETIME)<= paO2_datetime 
        ),
        paO2_fiO2_initial_rollup AS (
            SELECT 
                person_id, 
                admit_date, 
                paO2_datetime, 
                MIN(minutes_fiO2_paO2) As minutes_fiO2_paO2
            FROM paO2_fiO2_window 
            GROUP BY person_id, admit_date, paO2_datetime
        ),
        paO2_fiO2_initial_rollup_join AS (
            SELECT 
                initial_rollup.person_id, 
                initial_rollup.admit_date, 
                initial_rollup.paO2_datetime, 
                initial_rollup.minutes_fiO2_paO2,
                index_date, 
                fiO2, 
                fiO2_datetime, 
                paO2, 
                paO2fiO2_ratio 
            FROM paO2_fiO2_initial_rollup AS initial_rollup
            LEFT JOIN paO2_fiO2_window AS combined_window
            USING (person_id, paO2_datetime, minutes_fiO2_paO2)
        ),
        paO2_fiO2_rollup AS (
            SELECT 
                person_id, 
                CAST(admit_date AS DATE) as admit_date, 
                MIN(paO2fiO2_ratio) as min_paO2fiO2_ratio_prior
            FROM paO2_fiO2_initial_rollup_join 
            WHERE minutes_fiO2_paO2 <= 24*60
            GROUP BY person_id, admit_date
        ),
        spO2_window AS ( 
            SELECT 
                susp_inf_rollup.person_id, 
                min_bc, 
                min_systemic_abx, 
                susp_inf_rollup.admit_date, 
                index_date, 
                CAST(observation_datetime AS DATETIME) AS spO2_datetime, 
                meas_value as spO2, 
                datetime_diff(CAST(observation_datetime AS DATETIME), index_date, DAY) as days_spO2_index
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN spO2_from_flowsheet as flowsheet
            ON susp_inf_rollup.person_id = flowsheet.person_id
            WHERE
            DATE_SUB(CAST(index_date AS DATE), INTERVAL 2 DAY) > CAST(observation_datetime AS DATE) AND
            DATE_SUB(CAST(index_date AS DATE), INTERVAL 10 DAY) <= CAST(observation_datetime AS DATE) AND
            meas_value >0 AND meas_value <=100
        ),
        spO2_fiO2_window AS (
            SELECT 
                spO2_window.person_id, 
                spO2_window.admit_date, 
                spO2_window.index_date, 
                fiO2, 
                fiO2_datetime, 
                spO2, 
                spO2_datetime,  
                spO2/(NULLIF(fiO2, 0)) AS spO2fiO2_ratio, 
                datetime_diff(spO2_datetime, fiO2_datetime, MINUTE) as minutes_fiO2_spO2
            FROM fiO2_window AS fiO2_window
            INNER JOIN spO2_window AS spO2_window
            USING (person_id, index_date)
            WHERE CAST(fiO2_datetime AS DATETIME)<= spO2_datetime 
        ),
        spO2_fiO2_initial_rollup AS (
            SELECT 
                person_id, 
                admit_date, 
                spO2_datetime, 
                MIN(minutes_fiO2_spO2) As minutes_fiO2_spO2 
            FROM spO2_fiO2_window 
            GROUP BY person_id, admit_date, spO2_datetime
        ),
        spO2_fiO2_initial_rollup_join AS (
            SELECT 
                initial_rollup.person_id, 
                initial_rollup.admit_date, 
                initial_rollup.spO2_datetime, 
                initial_rollup.minutes_fiO2_spO2,
                index_date, 
                fiO2, 
                fiO2_datetime, 
                spO2, 
                spO2fiO2_ratio
            FROM spO2_fiO2_initial_rollup AS initial_rollup
            LEFT JOIN spO2_fiO2_window AS combined_window
            USING (person_id, spO2_datetime, minutes_fiO2_spO2)
        ),
        spO2_fiO2_rollup AS (
            SELECT 
                person_id, 
                CAST(admit_date AS DATE) as admit_date, 
                MIN(spO2fiO2_ratio) as min_spO2fiO2_ratio_prior
            FROM spO2_fiO2_initial_rollup_join 
            WHERE minutes_fiO2_spO2 <= 24*60
            GROUP BY person_id, admit_date
        ),
        glasgow_coma_scale_window AS (
            SELECT 
                susp_inf_rollup.*, 
                gcs.measurement_DATETIME AS gcs_datetime, 
                gcs.value_as_number AS glasgow_coma_scale,
                datetime_diff(gcs.measurement_DATETIME, index_date, DAY) as days_gcs_index
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN glasgow_coma_scale_from_measurement as gcs
            ON susp_inf_rollup.person_id =  gcs.person_id
            WHERE
            DATE_SUB(CAST(index_date AS DATE), INTERVAL 2 DAY) > CAST(measurement_DATETIME AS DATE) AND
            DATE_SUB(CAST(index_date AS DATE), INTERVAL 10 DAY) <= CAST(measurement_DATETIME AS DATE)
        ),
        glasgow_coma_scale_rollup AS (
            SELECT 
                person_id, 
                admit_date, 
                MIN(glasgow_coma_scale) as min_gcs_prior
            FROM glasgow_coma_scale_window 
            GROUP BY person_id, admit_date
        ),
        mean_arterial_pressure_window AS (
            SELECT 
                susp_inf_rollup.*, 
                mapm.measurement_DATETIME AS map_datetime,
                mapm.value_as_number AS map,
                datetime_diff(mapm.measurement_DATETIME, index_date, DAY) as days_map_index
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN mean_arterial_pressure_from_measurement as mapm
                ON susp_inf_rollup.person_id = mapm.person_id
            WHERE
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 2 DAY) > CAST(measurement_DATETIME AS DATE) AND
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 10 DAY) <= CAST(measurement_DATETIME AS DATE) AND
                mapm.value_as_number IS NOT NULL
        ),
        mean_arterial_pressure_rollup AS (
            SELECT 
                person_id, 
                admit_date, 
                MIN(map) as min_map_prior
            FROM mean_arterial_pressure_window 
            WHERE map >=10
            GROUP BY person_id, admit_date
        ),
        mech_vent_window AS (
            SELECT 
                susp_inf_rollup.*, 
                mech_vent.observation_datetime AS mech_vent_datetime, 
                mech_vent.meas_value as vent_mode,
                datetime_diff(mech_vent.observation_datetime, index_date, DAY) as days_mech_vent_index,
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN mech_vent_from_flowsheet AS mech_vent
            USING (person_id)
            WHERE
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 2 DAY) > CAST(CAST(observation_datetime AS DATETIME) AS DATE) AND
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 10 DAY) <= CAST(CAST(observation_datetime AS DATETIME) AS DATE) 
            ORDER BY  person_id, admit_date, index_date, observation_datetime 
        ),
        mech_vent_rollup AS (
            SELECT 
                person_id, 
                admit_date, 
                COUNT(vent_mode) as count_vent_mode_prior
            FROM mech_vent_window 
            GROUP BY person_id, admit_date
        ),
        urine_output_window AS (
            SELECT 
                susp_inf_rollup.*, 
                urine.measurement_DATETIME AS urine_datetime, 
                urine.value_as_number AS urine_volume,
                datetime_diff(measurement_DATETIME, index_date, DAY) as days_urine_index
            FROM {suspected_infection} AS susp_inf_rollup
            LEFT JOIN urine_from_measurement as urine
                ON susp_inf_rollup.person_id = urine.person_id 
            WHERE
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 2 DAY) > CAST(measurement_DATETIME AS DATE) AND
                DATE_SUB(CAST(index_date AS DATE), INTERVAL 10 DAY) <= CAST(measurement_DATETIME AS DATE) AND
                urine.value_as_number IS NOT NULL AND urine.value_as_number >=0
            ORDER BY person_id, admit_date, measurement_DATETIME
        ),
        admit_time AS (
            SELECT 
                person_id, 
                MIN(observation_datetime) AS ext_urine_datetime,
                EXTRACT(HOUR FROM MIN(observation_datetime)) AS hour, 
                (24-EXTRACT(HOUR FROM MIN(observation_datetime))) AS adjust_hours
            FROM urine_output_window AS urine
            LEFT JOIN som-nero-nigam-starr.jlemmon_explore.meas_vals_json AS flowsheets_orig  USING (person_id)
            WHERE CAST(admit_date AS DATE) = CAST(observation_datetime AS DATE)  
            AND observation_datetime <> DATETIME_TRUNC(observation_datetime, DAY)
            GROUP BY person_id
        ),
        discharge_time AS (
            SELECT 
                person_id, 
                MAX(observation_datetime) AS ext_urine_datetime,
                EXTRACT(HOUR FROM MAX(observation_datetime)) AS adjust_hours 
            FROM urine_output_window AS urine
            LEFT JOIN som-nero-nigam-starr.jlemmon_explore.meas_vals_json AS flowsheets_orig  USING (person_id)
            WHERE CAST(admit_date AS DATE) = CAST(observation_datetime AS DATE)  
            AND observation_datetime <> DATETIME_TRUNC(observation_datetime, DAY)
            GROUP BY person_id
        ),
        urine_output_initial_rollup AS (
            (
                SELECT 
                    person_id, 
                    admit_date, 
                    discharge_date, 
                    CAST(urine_datetime AS DATE) AS urine_date, 
                    SUM(urine_volume) as urine_daily_output_orig, 
                    SUM(urine_volume) as urine_daily_output_adj,
                    ext_urine_datetime, 
                    adjust_hours
                FROM urine_output_window 
                LEFT JOIN admit_time USING (person_id)
                WHERE CAST(urine_datetime AS DATE) <> CAST(admit_date AS DATE) AND CAST(urine_datetime AS DATE) <> CAST (discharge_date AS  DATE)
                GROUP BY person_id, admit_date, discharge_date, CAST (urine_datetime AS DATE), ext_urine_datetime, adjust_hours 
            )
            UNION ALL
            (
                SELECT 
                    person_id, 
                    admit_date, 
                    discharge_date, 
                    CAST(urine_datetime AS DATE) AS urine_date, 
                    SUM(urine_volume) as urine_daily_output_orig, 
                    (SUM(urine_volume))*24/adjust_hours as urine_daily_output_adj,
                    ext_urine_datetime, 
                    adjust_hours
                FROM urine_output_window 
                LEFT JOIN admit_time USING (person_id)
                WHERE CAST(urine_datetime AS DATE) = CAST(admit_date AS DATE) 
                GROUP BY person_id, admit_date, discharge_date, CAST(urine_datetime AS DATE), ext_urine_datetime, adjust_hours
            )
            UNION ALL
            (
                SELECT 
                    person_id, 
                    admit_date, 
                    discharge_date, 
                    CAST(urine_datetime AS DATE) AS urine_date, 
                    SUM(urine_volume) as urine_daily_output_orig, 
                    (SUM(urine_volume))*24/adjust_hours as urine_daily_output_adj,
                    ext_urine_datetime, 
                    adjust_hours
                FROM urine_output_window 
                LEFT JOIN discharge_time USING (person_id)
                WHERE CAST(urine_datetime AS DATE) = CAST(discharge_date AS DATE) AND adjust_hours <> 0 
                GROUP BY person_id, admit_date, discharge_date, CAST(urine_datetime AS DATE), ext_urine_datetime, adjust_hours
            )
            UNION ALL # THIS LAST BIT DEALS WITH DISCHARGE AT 0:00:00 HOURS
            (
                SELECT 
                    person_id, 
                    admit_date, 
                    discharge_date, 
                    CAST(urine_datetime AS DATE) AS urine_date, 
                    SUM(urine_volume) as urine_daily_output_orig, 
                    (SUM(urine_volume))*24 as urine_daily_output_adj,
                    ext_urine_datetime, 
                    adjust_hours
                FROM urine_output_window 
                LEFT JOIN discharge_time USING (person_id)
                WHERE CAST(urine_datetime AS DATE) = CAST(discharge_date AS DATE) AND adjust_hours = 0 
                GROUP BY person_id, admit_date, discharge_date, CAST(urine_datetime AS DATE), ext_urine_datetime, adjust_hours
            )
        ),
        urine_output_rollup AS (
            SELECT 
                person_id, 
                admit_date, 
                Min(urine_daily_output_adj) as min_urine_daily_prior
            FROM urine_output_initial_rollup 
            GROUP BY person_id, admit_date
        )
        # Begin sepsis and shock cohort
        SELECT 
            susp_inf_rollup.person_id, 
            CAST(susp_inf_rollup.admit_date AS DATE) AS admit_date, 
            CAST(discharge_date AS DATE) AS discharge_date,
            min_bc, 
            min_systemic_abx, 
            index_date, 
            min_platelet_prior, 
            CASE 
                WHEN min_platelet_prior IS NULL THEN 0 
                WHEN min_platelet_prior <20 THEN 4 
                WHEN min_platelet_prior < 50 THEN 3
                WHEN min_platelet_prior < 100 THEN 2 
                WHEN min_platelet_prior < 150 THEN 1 
                ELSE 0 
            END plat_SOFA_prior,
            max_bilirubin_prior, 
            CASE 
                WHEN max_bilirubin_prior IS NULL THEN 0 
                WHEN max_bilirubin_prior >= 12 THEN 4 
                WHEN max_bilirubin_prior >= 6 THEN 3
                WHEN max_bilirubin_prior >= 2 THEN 2 
                WHEN max_bilirubin_prior >= 1.2 THEN 1 
                ELSE 0 
            END bili_SOFA_prior,
            max_creatinine_prior, 
            CASE 
                WHEN max_creatinine_prior IS NULL THEN 0 
                WHEN max_creatinine_prior >= 5 THEN 4 
                WHEN max_creatinine_prior >= 3.5 THEN 3
                WHEN max_creatinine_prior >= 2 THEN 2 
                WHEN max_creatinine_prior >= 1.2 THEN 1 
                ELSE 0 
            END crea_SOFA_prior,
            max_vaso_days_prior, 
            min_map_prior, 
            CASE 
                WHEN max_vaso_days_prior IS NOT NULL THEN 2
                WHEN min_map_prior <70 THEN 1 
                ELSE 0 
            END cv_SOFA_prior,
            min_paO2fiO2_ratio_prior,
            min_spO2fiO2_ratio_prior, 
            count_vent_mode_prior, 
            CASE 
                WHEN (count_vent_mode_prior IS NOT NULL AND min_paO2fiO2_ratio_prior < 100) THEN 4
                WHEN count_vent_mode_prior IS NOT NULL THEN 3 
                WHEN min_paO2fiO2_ratio_prior < 300 THEN 2 
                WHEN min_paO2fiO2_ratio_prior < 400 THEN 1
                ELSE 0 
            END resp_SOFA_prior,
            min_gcs_prior, 
            CASE 
                WHEN min_gcs_prior IS NULL THEN 0 
                WHEN min_gcs_prior < 6 THEN 4 
                WHEN min_gcs_prior < 10 THEN 3
                WHEN min_gcs_prior < 13 THEN 2 
                WHEN min_gcs_prior < 15 THEN 1 
                ELSE 0 
            END gcs_SOFA_prior,
            min_urine_daily_prior, 
            CASE 
                WHEN min_urine_daily_prior IS NULL THEN 0 
                WHEN min_urine_daily_prior < 200 THEN 4 
                WHEN min_urine_daily_prior < 500 THEN 3 
                ELSE 0 
            END urine_SOFA_prior,
            CASE 
                WHEN max_vaso_days_prior IS NOT NULL THEN 1 
                ELSE 0 
            END vaso_shock_prior,
            CASE 
                WHEN min_map_prior < 65 THEN 1 
                ELSE 0 
            END map_shock_prior,
            max_lactate_prior, 
            CASE 
                WHEN max_lactate_prior > 2 THEN 1 
                ELSE 0 
            END lact_shock_prior, 
        FROM {suspected_infection} AS susp_inf_rollup
        LEFT JOIN platelet_rollup USING (person_id, admit_date)
        LEFT JOIN bilirubin_rollup USING (person_id, admit_date)
        LEFT JOIN creatinine_rollup USING (person_id, admit_date)
        LEFT JOIN vasopressor_rollup USING (person_id, admit_date)
        LEFT JOIN mean_arterial_pressure_rollup USING (person_id, admit_date)
        LEFT JOIN paO2_fiO2_rollup USING (person_id, admit_date)
        LEFT JOIN spO2_fiO2_rollup USING (person_id, admit_date)
        LEFT JOIN mech_vent_rollup USING (person_id, admit_date)
        LEFT JOIN glasgow_coma_scale_rollup USING (person_id, admit_date)
        LEFT JOIN urine_output_rollup USING (person_id, admit_date)
        LEFT JOIN lactate_rollup USING (person_id, admit_date)
        ORDER BY person_id, CAST(admit_date AS DATE)
        """
        if not format_query:
            return query
        else:
            return query.format_map(self.config_dict)

    def get_sepsis_cohort_combined(self, format_query=True):
        query = """
        SELECT 
            cohort.person_id, 
            cohort.admit_date, 
            cohort.discharge_date, 
            cohort.min_bc, 
            cohort.min_systemic_abx, 
            cohort.index_date, 
            cohort.min_platelet, 
            cohort.max_bilirubin, 
            cohort.max_creatinine, 
            cohort.max_vaso_days, 
            cohort.min_map, 
            cohort.min_paO2fiO2_ratio,
            cohort.min_spO2fiO2_ratio,
            cohort.count_vent_mode, 
            cohort.min_gcs, 
            cohort.min_urine_daily, 
            cohort.max_lactate,
            prior.min_platelet_prior, 
            prior.max_bilirubin_prior, 
            prior.max_creatinine_prior, 
            prior.max_vaso_days_prior, 
            prior.min_map_prior, 
            prior.min_paO2fiO2_ratio_prior,
            prior.min_spO2fiO2_ratio_prior,
            prior.count_vent_mode_prior, 
            prior.min_gcs_prior, 
            prior.min_urine_daily_prior, 
            prior.max_lactate_prior,
            plat_SOFA, 
            bili_SOFA, 
            crea_SOFA, 
            cv_SOFA, 
            resp_SOFA, 
            gcs_SOFA, 
            urine_SOFA, 
            (plat_SOFA + bili_SOFA + crea_SOFA + cv_SOFA + resp_SOFA + gcs_SOFA + urine_SOFA) AS SOFA_score_current,
            plat_SOFA_prior, 
            bili_SOFA_prior, 
            crea_SOFA_prior, 
            cv_SOFA_prior, 
            resp_SOFA_prior, 
            gcs_SOFA_prior, 
            urine_SOFA_prior,  
            (plat_SOFA_prior + bili_SOFA_prior + crea_SOFA_prior + cv_SOFA_prior + resp_SOFA_prior + gcs_SOFA_prior + urine_SOFA_prior) AS SOFA_score_prior,  
            ((plat_SOFA + bili_SOFA + crea_SOFA + cv_SOFA + resp_SOFA + gcs_SOFA + urine_SOFA) - 
             (plat_SOFA_prior + bili_SOFA_prior + crea_SOFA_prior + cv_SOFA_prior + resp_SOFA_prior + gcs_SOFA_prior + urine_SOFA_prior)) AS SOFA_score_diff,
            CASE 
                WHEN (plat_SOFA + bili_SOFA + crea_SOFA + cv_SOFA + resp_SOFA + gcs_SOFA + urine_SOFA) >=2 THEN "Yes" 
                ELSE "No" 
            END SOFA_current,
            CASE 
                WHEN (plat_SOFA_prior + bili_SOFA_prior + crea_SOFA_prior + cv_SOFA_prior + resp_SOFA_prior + gcs_SOFA_prior + urine_SOFA_prior) >=2 THEN "Yes" 
                ELSE "No" 
            END SOFA_prior,
            CASE 
                WHEN ((plat_SOFA + bili_SOFA + crea_SOFA + cv_SOFA + resp_SOFA + gcs_SOFA + urine_SOFA) - 
                      (plat_SOFA_prior + bili_SOFA_prior + crea_SOFA_prior + cv_SOFA_prior + resp_SOFA_prior + gcs_SOFA_prior + urine_SOFA_prior)) >=2 THEN "Yes" 
                ELSE "No" 
            END SOFA_diff,
            vaso_shock, 
            map_shock, 
            lact_shock, 
            (vaso_shock + map_shock + lact_shock) AS shock_score,
            vaso_shock_prior, 
            map_shock_prior, 
            lact_shock_prior, 
            (vaso_shock_prior + map_shock_prior + lact_shock_prior ) as shock_score_prior,
            # Modification for peds to remove mean arterial pressure
            CASE WHEN (vaso_shock + lact_shock) = 2 THEN "Yes" ELSE "No" END shock,  
            CASE WHEN (vaso_shock_prior + lact_shock_prior) = 2 THEN "Yes" ELSE "No" END shock_prior,  
            CASE WHEN ((vaso_shock + lact_shock) - (vaso_shock_prior + lact_shock_prior)) = 2 THEN "Yes" ELSE "No" END shock_diff, 

            CASE WHEN plat_SOFA >= 2 THEN 1 ELSE 0 END plat_SOFA_GT2,
            CASE WHEN bili_SOFA >= 2 THEN 1 ELSE 0 END bili_SOFA_GT2,
            CASE WHEN crea_SOFA >= 2 THEN 1 ELSE 0 END crea_SOFA_GT2,
            CASE WHEN cv_SOFA >= 2 THEN 1 ELSE 0 END cv_SOFA_GT2,
            CASE WHEN resp_SOFA >= 2 THEN 1 ELSE 0 END resp_SOFA_GT2,
            CASE WHEN gcs_SOFA >= 2 THEN 1 ELSE 0 END gcs_SOFA_GT2,
            CASE WHEN urine_SOFA >= 2 THEN 1 ELSE 0 END urine_SOFA_GT2,

            CASE WHEN plat_SOFA - plat_SOFA_prior >= 2 THEN 1 ELSE 0 END plat_SOFA_GT2_diff,
            CASE WHEN bili_SOFA -bili_SOFA_prior >= 2 THEN 1 ELSE 0 END bili_SOFA_GT2_diff,
            CASE WHEN crea_SOFA - crea_SOFA_prior >= 2 THEN 1 ELSE 0 END crea_SOFA_GT2_diff,
            CASE WHEN cv_SOFA - cv_SOFA_prior >= 2 THEN 1 ELSE 0 END cv_SOFA_GT2_diff,
            CASE WHEN resp_SOFA - resp_SOFA_prior >= 2 THEN 1 ELSE 0 END resp_SOFA_GT2_diff,
            CASE WHEN gcs_SOFA - gcs_SOFA_prior >= 2 THEN 1 ELSE 0 END gcs_SOFA_GT2_diff,
            CASE WHEN urine_SOFA - urine_SOFA_prior >= 2 THEN 1 ELSE 0 END urine_SOFA_GT2_diff,

            CASE WHEN vaso_shock - vaso_shock_prior = 1 THEN 1 ELSE 0 END vaso_shock_diff,
            CASE WHEN map_shock - map_shock_prior = 1 THEN 1 ELSE 0 END map_shock_diff,
            CASE WHEN lact_shock - lact_shock_prior = 1 THEN 1 ELSE 0 END lact_shock_diff
            FROM 
                {sepsis_initial} as cohort
            LEFT JOIN
                {sepsis_prior} as prior USING (person_id, admit_date)
            ORDER BY
                person_id, CAST(admit_date AS DATE)
        """
        if not format_query:
            return query
        else:
            return query.format_map(self.config_dict)

    def get_cohort_with_demographics(self, format_query=True):
        query = """
            WITH gender_name AS (
                SELECT 
                    person.*, 
                    concept.concept_name AS gender_name,
                FROM `{dataset_project}.{dataset}.person` AS person 
                LEFT JOIN `{dataset_project}.{dataset}.concept` AS concept ON person.gender_concept_id = concept.concept_id
            ),
            race_name AS (
                SELECT 
                    person.*, 
                    concept.concept_name AS race_name,
                FROM `{dataset_project}.{dataset}.person` AS person
                LEFT JOIN `{dataset_project}.{dataset}.concept` AS concept ON person.race_concept_id = concept.concept_id
            ),
            ethnicity_name AS (
                SELECT 
                    person.*, 
                    concept.concept_name AS ethnicity_name,
                FROM `{dataset_project}.{dataset}.person` AS person 
                LEFT JOIN `{dataset_project}.{dataset}.concept` AS concept ON person.ethnicity_concept_id = concept.concept_id
            )
            SELECT 
                cohort.*, 
                person.birth_DATETIME, 
                gender_name.gender_name, 
                race_name.race_name, 
                ethnicity_name.ethnicity_name, 
                DATE_DIFF(CAST(admit_date AS DATE), CAST(person.birth_DATETIME AS DATE), YEAR) AS age_in_years,
            CASE 
                WHEN DATE_DIFF(CAST(admit_date AS DATE), CAST(person.birth_DATETIME AS DATE), YEAR) <= 18 THEN 0
                ELSE 1 
            END adult_at_admission
            FROM {sepsis_combined} AS cohort
            LEFT JOIN gender_name USING (person_id) 
            LEFT JOIN race_name USING (person_id)
            LEFT JOIN ethnicity_name USING (person_id)
            LEFT JOIN `{dataset_project}.{dataset}.person` AS person USING (person_id)
            ORDER BY person_id, CAST(admit_date AS DATE)
        """
        if not format_query:
            return query
        else:
            return query.format_map(self.config_dict)

    def get_cohort_with_pediatric(self, format_query=True):
        query="""
            WITH 
            crea_withpeds_prior AS (
                (
                    SELECT person_id, admit_date, 
                        CASE 
                            WHEN max_creatinine_prior IS NULL THEN 0 
                            WHEN max_creatinine_prior >= 1.6 THEN 4 
                            WHEN max_creatinine_prior >= 1.2 THEN 3
                            WHEN max_creatinine_prior >= 1.0 THEN 2 
                            WHEN max_creatinine_prior >= 0.8 THEN 1 
                            ELSE 0 
                        END crea_SOFA_prior_update
                    FROM {sepsis_combined_demographics}
                    WHERE age_in_years*12 < 1
                )
                UNION ALL 
                (
                    SELECT 
                        person_id, 
                        admit_date, 
                        CASE 
                            WHEN max_creatinine_prior IS NULL THEN 0 
                            WHEN max_creatinine_prior >= 1.2 THEN 4 
                            WHEN max_creatinine_prior >= 0.8 THEN 3
                            WHEN max_creatinine_prior >= 0.5 THEN 2 
                            WHEN max_creatinine_prior >= 0.3 THEN 1 
                            ELSE 0 
                        END crea_SOFA_prior_update
                    FROM {sepsis_combined_demographics}
                    WHERE age_in_years*12 >= 1 AND age_in_years*12 < 12
                )
                UNION ALL 
                (
                    SELECT  
                        person_id, 
                        admit_date, 
                        CASE 
                            WHEN max_creatinine_prior IS NULL THEN 0 
                            WHEN max_creatinine_prior >= 1.5 THEN 4 
                            WHEN max_creatinine_prior >= 1.1 THEN 3
                            WHEN max_creatinine_prior >= 0.6 THEN 2 
                            WHEN max_creatinine_prior >= 0.4 THEN 1 
                            ELSE 0 
                        END crea_SOFA_prior_update
                    FROM {sepsis_combined_demographics}
                    WHERE age_in_years*12 >= 12 AND age_in_years*12 < 24
                )
                UNION ALL 
                (
                    SELECT  
                        person_id, 
                        admit_date, 
                        CASE 
                            WHEN max_creatinine_prior IS NULL THEN 0 
                            WHEN max_creatinine_prior >= 2.3 THEN 4 
                            WHEN max_creatinine_prior >= 1.6 THEN 3
                            WHEN max_creatinine_prior >= 0.9 THEN 2 
                            WHEN max_creatinine_prior >= 0.6 THEN 1 
                            ELSE 0 
                        END crea_SOFA_prior_update
                    FROM {sepsis_combined_demographics}
                    WHERE age_in_years*12 >= 24 AND age_in_years*12 < 60
                )
                UNION ALL 
                (
                    SELECT  
                        person_id, 
                        admit_date, 
                        CASE 
                            WHEN max_creatinine_prior IS NULL THEN 0 
                            WHEN max_creatinine_prior >= 2.6 THEN 4 
                            WHEN max_creatinine_prior >= 1.8 THEN 3
                            WHEN max_creatinine_prior >= 1.1 THEN 2 
                            WHEN max_creatinine_prior >= 0.7 THEN 1 
                            ELSE 0 
                        END crea_SOFA_prior_update
                    FROM {sepsis_combined_demographics}
                    WHERE age_in_years*12 >= 60 AND age_in_years*12 < 144
                )
                UNION ALL 
                (
                    SELECT  
                        person_id, 
                        admit_date, 
                        CASE 
                            WHEN max_creatinine_prior IS NULL THEN 0 
                            WHEN max_creatinine_prior >= 4.2 THEN 4 
                            WHEN max_creatinine_prior >= 2.9 THEN 3
                            WHEN max_creatinine_prior >= 1.7 THEN 2 
                            WHEN max_creatinine_prior >= 1.0 THEN 1 
                            ELSE 0
                        END crea_SOFA_prior_update
                    FROM {sepsis_combined_demographics}
                    WHERE age_in_years*12 >= 144 AND age_in_years*12 < 216
                )
                UNION ALL 
                (
                    SELECT  
                        person_id, 
                        admit_date, 
                        crea_SOFA_prior AS crea_SOFA_prior_update
                    FROM {sepsis_combined_demographics}
                    WHERE age_in_years*12 >= 216
                )
            ),
            crea_withpeds AS (
                SELECT  
                    person_id, 
                    admit_date, 
                    CASE 
                        WHEN max_creatinine IS NULL THEN 0 
                        WHEN max_creatinine >= 1.6 THEN 4 
                        WHEN max_creatinine >= 1.2 THEN 3
                        WHEN max_creatinine >= 1.0 THEN 2 
                        WHEN max_creatinine >= 0.8 THEN 1 
                        ELSE 0 
                    END crea_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 < 1
                UNION ALL 
                SELECT  
                    person_id, 
                    admit_date, 
                    CASE WHEN max_creatinine IS NULL THEN 0 
                    WHEN max_creatinine >= 1.2 THEN 4 
                    WHEN max_creatinine >= 0.8 THEN 3
                    WHEN max_creatinine >= 0.5 THEN 2 
                    WHEN max_creatinine >= 0.3 THEN 1 
                    ELSE 0 
                END crea_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 1 AND age_in_years*12 < 12
                UNION ALL 
                SELECT  
                    person_id, 
                    admit_date, 
                    CASE 
                        WHEN max_creatinine IS NULL THEN 0 
                        WHEN max_creatinine >= 1.5 THEN 4 
                        WHEN max_creatinine >= 1.1 THEN 3
                        WHEN max_creatinine >= 0.6 THEN 2 
                        WHEN max_creatinine >= 0.4 THEN 1 
                        ELSE 0 
                    END crea_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 12 AND age_in_years*12 < 24
                UNION ALL 
                SELECT  
                    person_id, 
                    admit_date, 
                    CASE 
                        WHEN max_creatinine IS NULL THEN 0 
                        WHEN max_creatinine >= 2.3 THEN 4 
                        WHEN max_creatinine >= 1.6 THEN 3
                        WHEN max_creatinine >= 0.9 THEN 2 
                        WHEN max_creatinine >= 0.6 THEN 1 
                        ELSE 0 
                    END crea_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 24 AND age_in_years*12 < 60
                UNION ALL 
                SELECT  
                    person_id, 
                    admit_date, 
                    CASE 
                        WHEN max_creatinine IS NULL THEN 0 
                        WHEN max_creatinine >= 2.6 THEN 4 
                        WHEN max_creatinine >= 1.8 THEN 3
                        WHEN max_creatinine >= 1.1 THEN 2 
                        WHEN max_creatinine >= 0.7 THEN 1 
                        ELSE 0 
                    END crea_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 60 AND age_in_years*12 < 144
                UNION ALL 
                SELECT  
                    person_id, 
                    admit_date, 
                    CASE 
                        WHEN max_creatinine IS NULL THEN 0 
                        WHEN max_creatinine >= 4.2 THEN 4 
                        WHEN max_creatinine >= 2.9 THEN 3
                        WHEN max_creatinine >= 1.7 THEN 2 
                        WHEN max_creatinine >= 1.0 THEN 1 
                        ELSE 0 
                    END crea_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 144 AND age_in_years*12 < 216
                UNION ALL 
                SELECT  
                    person_id, 
                    admit_date, 
                    crea_SOFA AS crea_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 216
            ),
            cv_withpeds_prior AS (
                SELECT 
                    person_id, 
                    admit_date,
                    CASE 
                        WHEN max_vaso_days_prior IS NOT NULL THEN 2 
                        WHEN min_map_prior <46 THEN 1 
                        ELSE 0 
                    END cv_SOFA_prior_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 < 1
                UNION ALL
                SELECT 
                    person_id, 
                    admit_date,
                    CASE 
                        WHEN max_vaso_days_prior IS NOT NULL THEN 2 
                        WHEN min_map_prior <55 THEN 1 
                        ELSE 0 END cv_SOFA_prior_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 1 AND age_in_years*12 < 12  
                UNION ALL
                SELECT 
                    person_id, 
                    admit_date,
                    CASE 
                        WHEN max_vaso_days_prior IS NOT NULL THEN 2 
                        WHEN min_map_prior <60 THEN 1 
                        ELSE 0 
                    END cv_SOFA_prior_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 12 AND age_in_years*12 < 24
                UNION ALL
                SELECT 
                    person_id, 
                    admit_date,
                    CASE 
                        WHEN max_vaso_days_prior IS NOT NULL THEN 2 
                        WHEN min_map_prior <62 THEN 1 
                        ELSE 0 
                    END cv_SOFA_prior_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 24 AND age_in_years*12 < 60
                UNION ALL
                SELECT 
                    person_id, 
                    admit_date,
                    CASE 
                        WHEN max_vaso_days_prior IS NOT NULL THEN 2 
                        WHEN min_map_prior <65 THEN 1 
                        ELSE 0 
                    END cv_SOFA_prior_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 60 AND age_in_years*12 < 144
                UNION ALL
                SELECT 
                    person_id, 
                    admit_date,
                    CASE 
                        WHEN max_vaso_days_prior IS NOT NULL THEN 2 
                        WHEN min_map_prior <67 THEN 1 
                        ELSE 0 
                    END cv_SOFA_prior_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 144 AND age_in_years*12 < 216
                UNION ALL 
                SELECT  
                    person_id, 
                    admit_date, 
                    cv_SOFA_prior AS cv_SOFA_prior_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 216
            ),
            cv_withpeds AS (
                SELECT 
                    person_id, 
                    admit_date,
                    CASE 
                        WHEN max_vaso_days IS NOT NULL THEN 2 
                        WHEN min_map <46 THEN 1 
                        ELSE 0 
                    END cv_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 < 1
                UNION ALL
                SELECT 
                    person_id, 
                    admit_date,
                    CASE 
                        WHEN max_vaso_days IS NOT NULL THEN 2 
                        WHEN min_map <55 THEN 1 
                        ELSE 0 
                    END cv_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 1 AND age_in_years*12 < 12  
                UNION ALL
                SELECT 
                    person_id, 
                    admit_date,
                    CASE 
                        WHEN max_vaso_days IS NOT NULL THEN 2 
                        WHEN min_map <60 THEN 1 
                        ELSE 0 
                    END cv_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 12 AND age_in_years*12 < 24
                UNION ALL
                SELECT 
                    person_id, 
                    admit_date,
                    CASE 
                        WHEN max_vaso_days IS NOT NULL THEN 2 
                        WHEN min_map <62 THEN 1 
                        ELSE 0 
                    END cv_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 24 AND age_in_years*12 < 60
                UNION ALL
                SELECT 
                    person_id, 
                    admit_date,
                    CASE 
                        WHEN max_vaso_days IS NOT NULL THEN 2 
                        WHEN min_map <65 
                        THEN 1 ELSE 0 
                    END cv_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 60 AND age_in_years*12 < 144
                UNION ALL
                SELECT 
                    person_id, 
                    admit_date,
                    CASE 
                        WHEN max_vaso_days IS NOT NULL THEN 2 
                        WHEN min_map <67 THEN 1 
                        ELSE 0 
                    END cv_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 144 AND age_in_years*12 < 216
                UNION ALL 
                SELECT  
                    person_id, 
                    admit_date, 
                    cv_SOFA AS cv_SOFA_update
                FROM {sepsis_combined_demographics}
                WHERE age_in_years*12 >= 216
            )
            SELECT *,

                # Modification to include pediatrics by removing urine output
                CASE 
                    WHEN (plat_SOFA + bili_SOFA + crea_SOFA_update + cv_SOFA_update + resp_SOFA + gcs_SOFA) >=2 THEN "Yes" 
                    ELSE "No" 
                END SOFA_current_update,
                CASE 
                    WHEN (plat_SOFA_prior + bili_SOFA_prior + crea_SOFA_prior_update + cv_SOFA_prior_update + resp_SOFA_prior + gcs_SOFA_prior) >=2 THEN "Yes" 
                    ELSE "No" 
                END SOFA_prior_update,
                CASE 
                    WHEN ((plat_SOFA + bili_SOFA + crea_SOFA_update + cv_SOFA_update + resp_SOFA + gcs_SOFA) - 
                          (plat_SOFA_prior + bili_SOFA_prior + crea_SOFA_prior_update + cv_SOFA_prior_update + resp_SOFA_prior + gcs_SOFA_prior)) >=2 THEN "Yes" 
                    ELSE "No" 
                END SOFA_diff_update,

                (plat_SOFA + bili_SOFA + crea_SOFA_update + cv_SOFA_update + resp_SOFA + gcs_SOFA) AS SOFA_score_current_update,
                (plat_SOFA_prior + bili_SOFA_prior + crea_SOFA_prior_update + cv_SOFA_prior_update + resp_SOFA_prior + gcs_SOFA_prior) AS SOFA_score_prior_update,  
                
                ((plat_SOFA + bili_SOFA + crea_SOFA_update + cv_SOFA_update + resp_SOFA + gcs_SOFA) - 
                 (plat_SOFA_prior + bili_SOFA_prior + crea_SOFA_prior_update + cv_SOFA_prior_update + resp_SOFA_prior + gcs_SOFA_prior)) AS SOFA_score_diff_update,

                CASE 
                    WHEN 
                        crea_SOFA_update >= 2 THEN 1 
                        ELSE 0 
                    END crea_SOFA_GT2_update,
                CASE 
                    WHEN cv_SOFA_update >= 2 THEN 1 
                    ELSE 0 
                END cv_SOFA_GT2_update,
                CASE 
                    WHEN crea_SOFA_update - crea_SOFA_prior_update >= 2 THEN 1 
                    ELSE 0 
                END crea_SOFA_GT2_diff_update,
                CASE 
                    WHEN cv_SOFA_update - cv_SOFA_prior_update >= 2 THEN 1 
                    ELSE 0 
                END cv_SOFA_GT2_diff_update,
            FROM {sepsis_combined_demographics}
            LEFT JOIN crea_withpeds_prior USING (person_id, admit_date)
            LEFT JOIN crea_withpeds USING (person_id, admit_date)
            LEFT JOIN cv_withpeds_prior USING (person_id, admit_date)
            LEFT JOIN cv_withpeds USING (person_id, admit_date)
        """
        if not format_query:
            return query
        else:
            return query.format_map(self.config_dict)

    def get_admission_sepsis_cohort(self, format_query=True):
        query = """
            SELECT 
                admit.person_id, 
                admit.admit_date, 
                admit.discharge_date,
                sepsis.min_bc, 
                sepsis.min_systemic_abx, 
                sepsis.index_date AS infection_date,
                sepsis.SOFA_diff_update, 
                sepsis.SOFA_current_update, 
                sepsis.SOFA_prior_update, 
                sepsis.SOFA_score_diff_update, 
                sepsis.SOFA_score_current_update, 
                sepsis.SOFA_score_prior_update,
                sepsis.shock_diff, 
                sepsis.shock, 
                sepsis.shock_prior,
                DATE_DIFF(CAST(admit.admit_date AS DATE), CAST(index_date AS DATE), DAY) AS days_admit_index,
                CASE 
                    WHEN index_date IS NULL THEN 0 
                    WHEN  DATE_DIFF(CAST(admit.admit_date AS DATE), CAST(index_date AS DATE), DAY) >= -3 THEN 1
                    ELSE 0 
                END early_infection, 
                CASE 
                    WHEN index_date IS NULL THEN 0 
                    WHEN  (DATE_DIFF(CAST(admit.admit_date AS DATE), CAST(index_date AS DATE), DAY) >= -3 AND SOFA_diff_update = "Yes") THEN 1
                ELSE 0 END early_sepsis,
                person.birth_DATETIME, 
                DATE_DIFF(CAST(admit.admit_date AS DATE), CAST(person.birth_DATETIME AS DATE), YEAR) AS age_in_years,
                CASE 
                    WHEN DATE_DIFF(CAST(admit.admit_date AS DATE), CAST(person.birth_DATETIME AS DATE), YEAR) <= 18 THEN 0 
                    ELSE 1 
                END adult_at_admission
            FROM {admission_rollup}  AS admit
            LEFT JOIN {sepsis_with_pediatric} AS sepsis
                ON (admit.person_id = sepsis.person_id AND CAST(admit.admit_date AS DATE) = CAST (sepsis.admit_DATE AS DATE))
            LEFT JOIN `{dataset_project}.{dataset}.person` AS person ON admit.person_id = person.person_id
            ORDER BY person_id, CAST(admit.admit_date AS DATE), index_date
        """
        if not format_query:
            return query
        else:
            return query.format_map(self.config_dict)

    def get_sampled_cohort(self, format_query=True):
        query = """
            SELECT * EXCEPT (rnd, pos), 
            FARM_FINGERPRINT(GENERATE_UUID()) as prediction_id
            FROM (
                SELECT *, ROW_NUMBER() OVER(PARTITION BY person_id ORDER BY rnd) AS pos
                FROM (
                    SELECT 
                        *,
                        FARM_FINGERPRINT(CONCAT(CAST(person_id AS STRING), CAST(admit_date AS STRING), CAST(discharge_date AS STRING))) as rnd
                    FROM {admission_sepsis_cohort}
                )
            )
            WHERE pos = 1
            ORDER BY person_id, admit_date
        """
        if not format_query:
            return query
        else:
            return query.format_map(self.config_dict)

In [72]:
config_dict = {
    "dataset_project": "som-nero-nigam-starr",
    "rs_dataset_project": "som-nero-nigam-starr",
    "dataset": "cdm_subset",
#     "dataset": "starr_omop_cdm5_deid_2022_08_01",
    "rs_dataset": "jlemmon_explore",
}
cohort_names = {
    "admission_rollup": "sepsis_admission_rollup",
    "suspected_infection": "susp_inf_rollup",
    "sepsis_initial": "sepsis_cohort_initial",
    "sepsis_prior": "sepsis_cohort_prior",
    "sepsis_combined": "sepsis_cohort_combined",
    "sepsis_combined_demographics": "sepsis_cohort_combined_demographics",
    "sepsis_with_pediatric": "sepsis_cohort_with_pediatric",
    "admission_sepsis_cohort": "admission_sepsis_cohort",
    "sepsis_cohort_final": "sepsis_cohort_final",
}
cohort_names_long = {
    key: "{rs_dataset_project}.{rs_dataset}.{cohort_name}".format(
        cohort_name=value, **config_dict
    )
    for key, value in cohort_names.items()
}
config_dict = {**config_dict, **cohort_names_long}

In [73]:
cohort = SepsisCohort(**config_dict)
admission_rollup_query = cohort.get_admission_rollup()
suspected_infection_query = cohort.get_suspected_infection()
sepsis_cohort_initial_query = cohort.get_sepsis_cohort_initial()
sepsis_cohort_prior_query = cohort.get_sepsis_cohort_prior()
sepsis_cohort_combined_query = cohort.get_sepsis_cohort_combined()
sepsis_cohort_combined_demographics_query = cohort.get_cohort_with_demographics()
sepsis_cohort_with_pediatric_query = cohort.get_cohort_with_pediatric()
admission_sepsis_cohort_query = cohort.get_admission_sepsis_cohort()
sampled_cohort_query = cohort.get_sampled_cohort()

/local-scratch/nigam/envs/jlemmon/conl/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [34]:
cohort.db.execute_sql_to_destination_table(
        query=admission_rollup_query, destination=config_dict["admission_rollup"]
    )
adm_ru_df = cohort.db.read_sql_query(
    """
            SELECT *
            FROM {admission_rollup}
        """.format_map(
        config_dict
    ),
    use_bqstorage_api=True,
)

In [35]:
adm_ru_df

,person_id,visit_concept_id,visit_start_datetime,visit_end_datetime
0,31040000,262,2018-09-08 17:00:00,2018-09-10 16:39:00
1,31040000,262,2018-07-01 16:46:00,2018-07-02 17:13:00
2,31040000,262,2018-11-01 17:13:00,2018-11-04 20:05:00
3,30779904,262,2017-01-08 12:59:00,2017-01-22 14:58:00
4,31125761,262,2012-01-24 23:51:00,2012-02-01 12:38:00
...,...,...,...,...
2954,30607103,9201,2010-05-31 07:20:00,2010-06-02 13:30:00
2955,32069119,9201,2020-09-30 11:59:00,2020-10-03 18:10:00
2956,30644479,9201,2012-12-06 05:55:00,2012-12-11 13:25:00
2957,31046911,9201,2020-02-09 06:09:00,2020-02-12 20:00:00


In [45]:
cohort.db.execute_sql_to_destination_table(
        query=suspected_infection_query, destination=config_dict["suspected_infection"]
    )
sus_inf_df = cohort.db.read_sql_query(
    """
            SELECT *
            FROM {suspected_infection}
        """.format_map(
        config_dict
    ),
    use_bqstorage_api=True,
)

In [46]:
sus_inf_df

,person_id,admit_date,discharge_date,min_bc,min_systemic_abx,index_date
0,29927014,2011-05-13 17:08:00,2011-05-14 15:13:00,2011-05-13 17:28:00,2011-05-13,2011-05-13 00:00:00
1,29927202,2020-03-31 22:24:00,2020-04-08 15:10:00,2020-03-31 20:34:00,2020-03-31,2020-03-31 00:00:00
2,29928008,2008-10-22 20:56:00,2008-10-25 15:14:00,2008-10-22 16:56:00,2008-10-22,2008-10-22 00:00:00
3,29928008,2019-06-13 22:52:00,2019-06-20 14:24:00,2019-06-13 23:28:00,2019-06-13,2019-06-13 00:00:00
4,29933334,2009-08-16 16:18:00,2009-08-17 14:29:00,2009-08-16 11:00:00,2009-08-16,2009-08-16 00:00:00
...,...,...,...,...,...,...
866,64672606,2021-05-24 23:07:00,2021-06-22 17:45:00,2021-05-24 23:16:00,2021-05-25,2021-05-24 23:16:00
867,64780781,2021-08-07 10:36:00,2021-08-09 18:30:00,2021-08-07 15:45:00,2021-08-07,2021-08-07 00:00:00
868,64781747,2021-04-09 09:19:00,2021-04-13 18:26:00,2021-04-09 09:35:00,2021-04-09,2021-04-09 00:00:00
869,72248433,2021-09-04 05:04:00,2021-10-23 14:04:00,2021-09-06 18:11:00,2021-09-05,2021-09-05 00:00:00


In [56]:
cohort.db.execute_sql_to_destination_table(
        query=sepsis_cohort_initial_query, destination=config_dict["sepsis_initial"]
    )
sep_init_df = cohort.db.read_sql_query(
    """
            SELECT *
            FROM {sepsis_initial}
        """.format_map(
        config_dict
    ),
    use_bqstorage_api=True,
)
# GET BIRTHDATE AND AGE AT ADMISSION

In [57]:
sep_init_df

,person_id,admit_date,discharge_date,min_bc,min_systemic_abx,index_date,min_platelet,plat_SOFA,max_bilirubin,bili_SOFA,...,count_vent_mode,resp_SOFA,min_gcs,gcs_SOFA,min_urine_daily,urine_SOFA,vaso_shock,map_shock,max_lactate,lact_shock
0,29927014,2011-05-13,2011-05-14,2011-05-13 17:28:00,2011-05-13,2011-05-13 00:00:00,60.0,2,0.9,0,...,<NA>,0,NaN,0,3078.260870,0,0,0,NaN,0
1,29927202,2020-03-31,2020-04-08,2020-03-31 20:34:00,2020-03-31,2020-03-31 00:00:00,243.0,0,NaN,0,...,<NA>,0,15.0,0,400.000000,3,0,0,NaN,0
2,29928008,2008-10-22,2008-10-25,2008-10-22 16:56:00,2008-10-22,2008-10-22 00:00:00,75.0,2,0.5,0,...,<NA>,0,NaN,0,1181.000000,0,0,0,2.47,1
3,29928008,2019-06-13,2019-06-20,2019-06-13 23:28:00,2019-06-13,2019-06-13 00:00:00,163.0,0,0.3,0,...,<NA>,0,15.0,0,1350.000000,0,0,0,5.64,1
4,29933334,2009-08-16,2009-08-17,2009-08-16 11:00:00,2009-08-16,2009-08-16 00:00:00,138.0,1,0.5,0,...,<NA>,0,NaN,0,1200.000000,0,0,0,2.73,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,64672606,2021-05-24,2021-06-22,2021-05-24 23:16:00,2021-05-25,2021-05-24 23:16:00,48.0,3,4.8,2,...,<NA>,0,14.0,1,NaN,0,1,1,1.90,0
867,64780781,2021-08-07,2021-08-09,2021-08-07 15:45:00,2021-08-07,2021-08-07 00:00:00,60.0,2,0.4,0,...,<NA>,0,15.0,0,417.391304,3,0,0,1.50,0
868,64781747,2021-04-09,2021-04-13,2021-04-09 09:35:00,2021-04-09,2021-04-09 00:00:00,317.0,0,0.7,0,...,3,3,NaN,0,2775.000000,0,0,0,0.90,0
869,72248433,2021-09-04,2021-10-23,2021-09-06 18:11:00,2021-09-05,2021-09-05 00:00:00,25.0,3,1.4,1,...,352,3,3.0,4,1040.000000,0,1,1,6.70,1


In [64]:
cohort.db.execute_sql_to_destination_table(
        query=sepsis_cohort_prior_query, destination=config_dict["sepsis_prior"]
    )
sep_pri_df = cohort.db.read_sql_query(
    """
            SELECT *
            FROM {sepsis_prior}
        """.format_map(
        config_dict
    ),
    use_bqstorage_api=True,
)

In [65]:
sep_pri_df

,person_id,admit_date,discharge_date,min_bc,min_systemic_abx,index_date,min_platelet_prior,plat_SOFA_prior,max_bilirubin_prior,bili_SOFA_prior,...,count_vent_mode_prior,resp_SOFA_prior,min_gcs_prior,gcs_SOFA_prior,min_urine_daily_prior,urine_SOFA_prior,vaso_shock_prior,map_shock_prior,max_lactate_prior,lact_shock_prior
0,29927014,2011-05-13,2011-05-14,2011-05-13 17:28:00,2011-05-13,2011-05-13 00:00:00,NaN,0,NaN,0,...,<NA>,0,NaN,0,NaN,0,0,0,NaN,0
1,29927202,2020-03-31,2020-04-08,2020-03-31 20:34:00,2020-03-31,2020-03-31 00:00:00,NaN,0,NaN,0,...,<NA>,0,NaN,0,NaN,0,0,0,NaN,0
2,29928008,2008-10-22,2008-10-25,2008-10-22 16:56:00,2008-10-22,2008-10-22 00:00:00,NaN,0,NaN,0,...,<NA>,0,NaN,0,NaN,0,0,0,NaN,0
3,29928008,2019-06-13,2019-06-20,2019-06-13 23:28:00,2019-06-13,2019-06-13 00:00:00,NaN,0,NaN,0,...,<NA>,0,NaN,0,NaN,0,0,0,NaN,0
4,29933334,2009-08-16,2009-08-17,2009-08-16 11:00:00,2009-08-16,2009-08-16 00:00:00,NaN,0,NaN,0,...,<NA>,0,NaN,0,NaN,0,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,64672606,2021-05-24,2021-06-22,2021-05-24 23:16:00,2021-05-25,2021-05-24 23:16:00,115.0,1,4.1,2,...,<NA>,0,15.0,0,0.0,4,1,1,NaN,0
867,64780781,2021-08-07,2021-08-09,2021-08-07 15:45:00,2021-08-07,2021-08-07 00:00:00,NaN,0,NaN,0,...,<NA>,0,NaN,0,NaN,0,0,0,NaN,0
868,64781747,2021-04-09,2021-04-13,2021-04-09 09:35:00,2021-04-09,2021-04-09 00:00:00,NaN,0,NaN,0,...,<NA>,0,NaN,0,NaN,0,0,0,NaN,0
869,72248433,2021-09-04,2021-10-23,2021-09-06 18:11:00,2021-09-05,2021-09-05 00:00:00,NaN,0,NaN,0,...,<NA>,0,NaN,0,NaN,0,0,0,NaN,0


In [66]:
cohort.db.execute_sql_to_destination_table(
    query=sepsis_cohort_combined_query, destination=config_dict["sepsis_combined"]
)
sep_com_df = cohort.db.read_sql_query(
    """
            SELECT *
            FROM {sepsis_combined}
        """.format_map(
        config_dict
    ),
    use_bqstorage_api=True,
)

In [67]:
sep_com_df

,person_id,admit_date,discharge_date,min_bc,min_systemic_abx,index_date,min_platelet,max_bilirubin,max_creatinine,max_vaso_days,...,plat_SOFA_GT2_diff,bili_SOFA_GT2_diff,crea_SOFA_GT2_diff,cv_SOFA_GT2_diff,resp_SOFA_GT2_diff,gcs_SOFA_GT2_diff,urine_SOFA_GT2_diff,vaso_shock_diff,map_shock_diff,lact_shock_diff
0,29927014,2011-05-13,2011-05-14,2011-05-13 17:28:00,2011-05-13,2011-05-13 00:00:00,60.0,0.9,0.80,<NA>,...,1,0,0,0,0,0,0,0,0,0
1,29927202,2020-03-31,2020-04-08,2020-03-31 20:34:00,2020-03-31,2020-03-31 00:00:00,243.0,NaN,0.79,<NA>,...,0,0,0,0,0,0,1,0,0,0
2,29928008,2008-10-22,2008-10-25,2008-10-22 16:56:00,2008-10-22,2008-10-22 00:00:00,75.0,0.5,1.20,<NA>,...,1,0,0,0,0,0,0,0,0,1
3,29928008,2019-06-13,2019-06-20,2019-06-13 23:28:00,2019-06-13,2019-06-13 00:00:00,163.0,0.3,0.68,<NA>,...,0,0,0,0,0,0,0,0,0,1
4,29933334,2009-08-16,2009-08-17,2009-08-16 11:00:00,2009-08-16,2009-08-16 00:00:00,138.0,0.5,1.40,<NA>,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,64672606,2021-05-24,2021-06-22,2021-05-24 23:16:00,2021-05-25,2021-05-24 23:16:00,48.0,4.8,4.16,1,...,1,0,0,0,0,0,0,0,0,0
871,64780781,2021-08-07,2021-08-09,2021-08-07 15:45:00,2021-08-07,2021-08-07 00:00:00,60.0,0.4,1.86,<NA>,...,1,0,0,0,0,0,1,0,0,0
872,64781747,2021-04-09,2021-04-13,2021-04-09 09:35:00,2021-04-09,2021-04-09 00:00:00,317.0,0.7,0.86,<NA>,...,0,0,0,0,1,0,0,0,0,0
873,72248433,2021-09-04,2021-10-23,2021-09-06 18:11:00,2021-09-05,2021-09-05 00:00:00,25.0,1.4,2.23,26,...,1,0,1,1,1,1,0,1,1,1


In [18]:
cohort.db.execute_sql_to_destination_table(
    query=sepsis_cohort_combined_demographics_query,
    destination=config_dict["sepsis_combined_demographics"],
)
sep_dem_df = cohort.db.read_sql_query(
    """
            SELECT *
            FROM {sepsis_combined_demographics}
        """.format_map(
        config_dict
    ),
    use_bqstorage_api=True,
)

In [26]:
sep_dem_df

,person_id,admit_date,discharge_date,min_bc,min_systemic_abx,index_date,min_platelet,max_bilirubin,max_creatinine,max_vaso_days,...,urine_SOFA_GT2_diff,vaso_shock_diff,map_shock_diff,lact_shock_diff,birth_DATETIME,gender_name,race_name,ethnicity_name,age_in_years,adult_at_admission
0,29923165,2015-01-29,2015-02-02,2015-01-29 09:53:00,2015-01-29,2015-01-29 00:00:00,NaN,NaN,0.70,<NA>,...,1,0,0,0,2015-01-29 00:00:00,FEMALE,No matching concept,Hispanic or Latino,0,0
1,29923194,2020-09-02,2020-09-03,2020-09-02 07:40:00,2020-09-02,2020-09-02 00:00:00,NaN,0.4,0.42,<NA>,...,0,0,0,0,1994-05-04 00:00:00,FEMALE,No matching concept,Hispanic or Latino,26,1
2,29923298,2018-12-27,2019-01-20,2019-01-02 13:33:00,2019-01-02,2019-01-02 00:00:00,NaN,NaN,0.31,14,...,0,0,0,0,2018-12-12 00:00:00,FEMALE,No matching concept,Hispanic or Latino,0,0
3,29923298,2019-08-23,2019-09-29,2019-08-29 16:27:00,2019-08-28,2019-08-28 00:00:00,NaN,0.4,0.25,11,...,0,0,0,0,2018-12-12 00:00:00,FEMALE,No matching concept,Hispanic or Latino,1,0
4,29923307,2016-12-14,2017-01-17,2017-01-05 16:59:00,2017-01-05,2017-01-05 00:00:00,NaN,NaN,0.20,<NA>,...,0,0,0,0,2016-12-13 00:00:00,FEMALE,No matching concept,Hispanic or Latino,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106475,86284569,2022-08-06,2022-08-09,2022-08-06 19:27:00,2022-08-06,2022-08-06 00:00:00,NaN,12.8,0.41,<NA>,...,0,0,0,0,2022-08-02 00:00:00,MALE,Asian,Not Hispanic or Latino,0,0
106476,86284849,2022-07-03,2022-07-09,2022-07-03 16:23:00,2022-07-03,2022-07-03 00:00:00,NaN,NaN,0.83,<NA>,...,0,0,0,0,1960-05-28 00:00:00,MALE,No matching concept,Hispanic or Latino,62,1
106477,86285320,2022-07-13,2022-07-19,2022-07-16 22:50:00,2022-07-17,2022-07-16 22:50:00,NaN,12.4,NaN,<NA>,...,0,0,0,0,2022-07-13 00:00:00,MALE,No matching concept,Hispanic or Latino,0,0
106478,86285959,2022-07-31,2022-08-02,2022-08-02 03:16:00,2022-08-02,2022-08-02 00:00:00,NaN,NaN,NaN,<NA>,...,0,0,0,0,2022-07-31 22:11:00,MALE,No matching concept,Hispanic or Latino,0,0


In [19]:
cohort.db.execute_sql_to_destination_table(
    query=sepsis_cohort_with_pediatric_query,
    destination=config_dict["sepsis_with_pediatric"],
)
sep_ped_df = cohort.db.read_sql_query(
    """
            SELECT *
            FROM {sepsis_with_pediatric}
        """.format_map(
        config_dict
    ),
    use_bqstorage_api=True,
)

In [27]:
sep_ped_df

,person_id,admit_date,discharge_date,min_bc,min_systemic_abx,index_date,min_platelet,max_bilirubin,max_creatinine,max_vaso_days,...,SOFA_current_update,SOFA_prior_update,SOFA_diff_update,SOFA_score_current_update,SOFA_score_prior_update,SOFA_score_diff_update,crea_SOFA_GT2_update,cv_SOFA_GT2_update,crea_SOFA_GT2_diff_update,cv_SOFA_GT2_diff_update
0,43999593,2020-08-10,2020-08-11,2020-08-10 22:22:00,2020-08-09,2020-08-09 00:00:00,NaN,0.2,0.19,<NA>,...,No,No,No,0,<NA>,<NA>,0,0,0,0
1,45689935,2020-09-03,2020-09-04,2020-09-02 22:22:00,2020-09-02,2020-09-02 00:00:00,NaN,NaN,NaN,<NA>,...,No,No,No,0,<NA>,<NA>,0,0,0,0
2,68467727,2021-06-03,2021-06-05,2021-06-03 02:14:00,2021-06-03,2021-06-03 00:00:00,NaN,NaN,NaN,<NA>,...,No,No,No,0,<NA>,<NA>,0,0,0,0
3,68506847,2021-07-17,2021-07-19,2021-07-17 17:32:00,2021-07-19,2021-07-17 17:32:00,NaN,NaN,0.20,<NA>,...,No,No,No,0,<NA>,<NA>,0,0,0,0
4,69122515,2021-10-01,2021-10-02,2021-09-30 03:21:00,2021-10-01,2021-09-30 03:21:00,NaN,0.3,0.24,<NA>,...,No,No,No,0,<NA>,<NA>,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106475,31920846,2019-03-29,2019-04-19,2019-04-01 09:09:00,2019-03-31,2019-03-31 00:00:00,NaN,3.3,3.66,11,...,Yes,No,Yes,15,0,15,1,1,1,1
106476,31794999,2020-04-01,2020-04-04,2020-04-03 04:05:00,2020-04-02,2020-04-02 00:00:00,NaN,2.9,7.86,1,...,Yes,Yes,Yes,16,5,11,1,1,0,0
106477,32029924,2019-02-20,2019-03-03,2019-02-22 12:29:00,2019-02-21,2019-02-21 00:00:00,NaN,9.0,9.40,2,...,Yes,Yes,Yes,16,5,11,1,1,0,1
106478,43725075,2019-11-04,2019-11-26,2019-11-12 18:11:00,2019-11-11,2019-11-11 00:00:00,NaN,19.7,4.10,16,...,Yes,Yes,No,17,17,0,1,1,0,0


In [20]:
cohort.db.execute_sql_to_destination_table(
    query=admission_sepsis_cohort_query,
    destination=config_dict["admission_sepsis_cohort"],
)
adm_sep_df = cohort.db.read_sql_query(
    """
            SELECT *
            FROM {admission_sepsis_cohort}
        """.format_map(
        config_dict
    ),
    use_bqstorage_api=True,
)

In [28]:
adm_sep_df

,person_id,admit_date,discharge_date,min_bc,min_systemic_abx,infection_date,SOFA_diff_update,SOFA_current_update,SOFA_prior_update,SOFA_score_diff_update,...,SOFA_score_prior_update,shock_diff,shock,shock_prior,days_admit_index,early_infection,early_sepsis,birth_DATETIME,age_in_years,adult_at_admission
0,29923435,2015-09-05,2015-09-14,2015-09-05 01:25:00,2015-09-05,2015-09-05 00:00:00,No,Yes,No,<NA>,...,<NA>,No,No,No,0,1,0,2015-09-05,0,0
1,29923933,2017-08-23,2017-09-16,2017-08-26 23:21:00,2017-08-28,2017-08-26 23:21:00,Yes,Yes,Yes,2,...,5,No,No,No,-3,1,1,2017-08-23,0,0
2,29924047,2019-05-03,2019-05-19,2019-05-03 10:42:00,2019-05-03,2019-05-03 00:00:00,Yes,Yes,No,9,...,0,No,No,No,0,1,1,1945-10-04,74,1
3,29927021,2015-03-22,2015-05-04,2015-04-03 11:36:00,2015-04-02,2015-04-02 00:00:00,No,Yes,Yes,-3,...,10,No,No,No,-11,0,0,2001-03-04,14,0
4,29927109,2015-12-15,2016-01-10,2015-12-18 07:12:00,2015-12-21,2015-12-18 07:12:00,Yes,Yes,No,7,...,1,No,No,No,-3,1,1,2015-05-21,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517471,80892069,2022-07-04,2022-07-10,2022-07-04 02:53:00,2022-07-04,2022-07-04 00:00:00,Yes,Yes,No,6,...,0,No,No,No,0,1,1,1949-01-28,73,1
517472,80897031,2022-06-21,2022-06-29,2022-06-23 10:42:00,2022-06-22,2022-06-22 00:00:00,Yes,Yes,No,6,...,0,No,No,No,-1,1,1,1979-05-30,43,1
517473,80975698,2022-07-06,2022-07-28,2022-07-13 00:54:00,2022-07-13,2022-07-13 00:00:00,Yes,Yes,Yes,2,...,4,No,Yes,Yes,-7,0,0,1980-03-09,42,1
517474,81038054,2022-06-03,2022-06-04,2022-06-03 23:51:00,2022-06-03,2022-06-03 00:00:00,Yes,Yes,No,6,...,0,Yes,Yes,No,0,1,1,1941-03-30,81,1


In [21]:
cohort.db.execute_sql_to_destination_table(
    query=sampled_cohort_query, destination=config_dict["sepsis_cohort_final"]
)
cohort_df = cohort.db.read_sql_query(
    """
            SELECT *
            FROM {sepsis_cohort_final}
        """.format_map(
        config_dict
    ),
    use_bqstorage_api=True,
)

In [29]:
cohort_df

,person_id,admit_date,discharge_date,min_bc,min_systemic_abx,infection_date,SOFA_diff_update,SOFA_current_update,SOFA_prior_update,SOFA_score_diff_update,...,shock_diff,shock,shock_prior,days_admit_index,early_infection,early_sepsis,birth_DATETIME,age_in_years,adult_at_admission,prediction_id
0,29923435,2015-09-05,2015-09-14,2015-09-05 01:25:00,2015-09-05,2015-09-05 00:00:00,No,Yes,No,<NA>,...,No,No,No,0,1,0,2015-09-05,0,0,-6102394233917083572
1,29923933,2017-08-23,2017-09-16,2017-08-26 23:21:00,2017-08-28,2017-08-26 23:21:00,Yes,Yes,Yes,2,...,No,No,No,-3,1,1,2017-08-23,0,0,2792807757321791817
2,29924047,2019-05-03,2019-05-19,2019-05-03 10:42:00,2019-05-03,2019-05-03 00:00:00,Yes,Yes,No,9,...,No,No,No,0,1,1,1945-10-04,74,1,-8749738461111544633
3,29927384,2019-05-21,2019-06-03,2019-05-23 10:45:00,2019-05-23,2019-05-23 00:00:00,Yes,Yes,Yes,8,...,No,No,No,-2,1,1,2014-03-16,5,0,-5547958901194810162
4,29928003,2015-03-18,2015-04-17,2015-03-19 06:17:00,2015-03-18,2015-03-18 00:00:00,Yes,Yes,No,7,...,No,No,No,0,1,1,2012-09-15,3,0,6626118506260432338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314211,80892069,2022-07-04,2022-07-10,2022-07-04 02:53:00,2022-07-04,2022-07-04 00:00:00,Yes,Yes,No,6,...,No,No,No,0,1,1,1949-01-28,73,1,-3160279584347972311
314212,80897031,2022-06-21,2022-06-29,2022-06-23 10:42:00,2022-06-22,2022-06-22 00:00:00,Yes,Yes,No,6,...,No,No,No,-1,1,1,1979-05-30,43,1,-2168604788320547015
314213,80975698,2022-07-06,2022-07-28,2022-07-13 00:54:00,2022-07-13,2022-07-13 00:00:00,Yes,Yes,Yes,2,...,No,Yes,Yes,-7,0,0,1980-03-09,42,1,2850855576965997076
314214,81038054,2022-06-03,2022-06-04,2022-06-03 23:51:00,2022-06-03,2022-06-03 00:00:00,Yes,Yes,No,6,...,Yes,Yes,No,0,1,1,1941-03-30,81,1,2420870947704829302
